In [1]:
import pandas as pd
from saveAndLoad import *
from aa_util import find_replacement_substring
from tqdm import tqdm
import math, subprocess
from Bio import SwissProt
from io import StringIO
import re
import numpy as np
import ast

# Create training data

In [2]:
clinvar_ref = pd.read_csv('/data/dandreas/clinvar/ref_sequences_clean.csv')
clinvar_ref_dict = clinvar_ref.set_index(['HGNC_ID', 'Assembly']).T.to_dict()
clinvar_mut = pd.read_csv('/data/dandreas/clinvar/clinvar_cancer_mutations_certain_pathogenicity.csv')
interpro_descriptions = pickleLoad('/data/dandreas/MutationModel/data/interpro_descriptions.pkl')
protein2ipr_human = pd.read_csv('./data/protein2ipr_human.tsv', sep='\t')
hgnc_uniprot_name_map = pickleLoad('/home/dandreas/SomaticMutationLLM/hgnc_names/hgnc_uniprot_name_map.pkl')

loading data from /data/dandreas/MutationModel/data/interpro_descriptions.pkl
loading data from /home/dandreas/SomaticMutationLLM/hgnc_names/hgnc_uniprot_name_map.pkl


In [3]:
def safe_pairs(x):
    if isinstance(x, list): return x
    if pd.isna(x): return []
    try:
        v = ast.literal_eval(str(x))
        return v if isinstance(v, list) else []
    except Exception:
        return []

def regions_text_simple(pairs, max_items=3):
    pairs = safe_pairs(pairs)
    shorts = [s for _, s in pairs[:max_items] if s]
    if not shorts:
        return ""
    k_total = len(pairs)
    return f"regions(n={k_total}): " + "; ".join(shorts)

def regions_text_detailed(pairs, interpro_descriptions):
    """
    Returns: '<Short1>: <Long1>; <Short2>: <Long2>; ...'
    - includes up to max_items regions
    - if no long description, emits just '<Short>'
    """
    pairs = safe_pairs(pairs)[:]
    parts = []
    for iid, short in pairs:
        short = (short or "").strip()
        long = interpro_descriptions.get(iid, "")
        if long:
            parts.append(f"{short}: {long}" if short else long)
        else:
            if short:
                parts.append(short)
    return "; ".join(parts) if parts else ""


def get_ref_seq(row, ref_dict):
    rec = ref_dict.get((row["HGNC_ID"], row["Assembly"]), {})
    return str(rec.get("Sequence", ""))

def size_bucket(m):
    if pd.isna(m): return "unknown"
    m = int(m)
    if m == 1: return "small"
    if 2 <= m <= 5: return "medium"
    return "large"

def clean_phenotypes(s):
    """
    Split on '|' (and tolerate ';'), strip, drop 'not provided' (case/diacritics insensitive),
    dedupe preserving order. Return '' if nothing left.
    """
    if pd.isna(s) or not str(s).strip():
        return ""
    raw = str(s).replace(";", "|").split("|")
    out = []
    seen = set()
    for item in raw:
        t = item.strip()
        if not t:
            continue
        lower = t.casefold()
        if lower == "not provided":
            continue
        if lower not in seen:
            seen.add(lower)
            out.append(t)
    return "; ".join(out)

def describe_row(row, ref_dict, interpro_descriptions, simple=False):
    gene = row.get("GeneSymbol") or str(row.get("HGNC_ID"))
    clin = str(row.get("ClinicalSignificance", "")).strip()
    phen_clean = row.get("_PhenotypesClean", "")
    mut_type = str(row.get("MutationType", "")).strip()

    mut_seq = str(row.get("Sequence", "")).strip()
    ref_seq = get_ref_seq(row, ref_dict)
    m = row.get("Levenshtein", float("nan"))
    dlen = len(mut_seq) - len(ref_seq)

    # NEW: per-region paired details (or simple list)
    if simple:
        rtxt = regions_text_simple(row.get("interpro_affected"), max_items=3)
    else:
        rtxt = regions_text_detailed(row.get("interpro_affected"), interpro_descriptions)

    phen_clause = f"; Phenotypes: {phen_clean}" if phen_clean else ""

    # frameshift / nonsense
    if mut_type.startswith("Frame_Shift") or mut_type.startswith("Nonsense"):
        reason = "frameshift" if mut_type.startswith("Frame_Shift") else "premature stop"
        return f"loss-of-function ({reason}) in {gene}; Clinical Significance: {clin}{phen_clause}; {rtxt}"

    # in-frame
    if mut_type == "In_Frame_Insertion":
        k = abs(dlen)
        return f"in-frame insertion of {k} aa in {gene}; Clinical Significance: {clin}{phen_clause}; {rtxt}"

    if mut_type == "In_Frame_Deletion":
        k = abs(dlen)
        return f"in-frame deletion of {k} aa in {gene}; Clinical Significance: {clin}{phen_clause}; {rtxt}"

    # substitution
    sb = size_bucket(m)
    m_disp = "unknown" if pd.isna(m) else f"{int(m)}"
    return f"substitution ({sb}; {m_disp} aa) in {gene}; Clinical Significance: {clin}{phen_clause}; {rtxt}"

def compute_diff_indices(mut_seq: str, ref_seq: str):
    out = find_replacement_substring(mut_seq, ref_seq)
    if out == (None, None, None):
        return np.nan, np.nan, np.nan
    start, end_mut, end_ref = out
    # columns requested: start, ref_end, mut_end
    return start, end_ref, end_mut

def crop_by_start(mut_seq: str, ref_seq: str, center, window_len: int = 2048):
    """
    Mutant: center at `center` (affected_aa_start); if right edge would
    exceed the mutant length, shift left to keep full window where possible.

    Reference: 
      - if len(ref) <= window_len: keep full reference (no crop)
      - else: left-align with the mutant window (use the mutant crop's start index),
              clamped so the ref crop stays within [0, len(ref)].
    Returns: (mut_crop, ref_crop)
    """
    Lm, Lr = len(mut_seq), len(ref_seq)

    # center fallback
    if pd.isna(center):
        center = Lm // 2
    center = int(center)

    # ---- Mutant crop (unchanged logic) ----
    half_left  = window_len // 2
    half_right = window_len - half_left

    m_end = min(Lm, center + half_right)
    m_start = m_end - window_len
    if m_start < 0:
        m_start = 0
        m_end = min(Lm, window_len)

    mut_crop = mut_seq[m_start:m_end]

    # ---- Reference crop (new logic) ----
    if Lr <= window_len:
        # keep full reference if it fits
        r_start, r_end = 0, Lr
    else:
        # left-align to mutant crop start, but keep within bounds
        r_start = max(0, min(m_start, Lr - window_len))
        r_end = r_start + window_len

    ref_crop = ref_seq[r_start:r_end]

    return mut_crop, ref_crop


# ---- integrate into your builder ----
def build_mutation_descriptions(df, ref_dict, interpro_descriptions, window_size: int = 1024):
    df = df.copy()

    # clean phenotypes (your existing helper)
    df["_PhenotypesClean"] = df["PhenotypeList"].apply(clean_phenotypes)

    # sequences
    df["_mut_seq"] = df["Sequence"].astype(str)
    df["_ref_seq"] = df.apply(lambda r: get_ref_seq(r, ref_dict), axis=1).astype(str)

    # --- compute and save indices ---
    idx_triplets = df.progress_apply(
        lambda r: compute_diff_indices(r["_mut_seq"], r["_ref_seq"]), axis=1
    )
    df[["affected_aa_start", "affected_ref_aa_end", "affected_mut_aa_end"]] = pd.DataFrame(
        idx_triplets.tolist(), index=df.index
    )

    # --- crop using your rule (same window on ref) ---
    crops = df.progress_apply(
        lambda r: crop_by_start(r["_mut_seq"], r["_ref_seq"], r["affected_aa_start"], window_len=window_size),
        axis=1
    )
    crop_df = pd.DataFrame(crops.tolist(), columns=["cropped mut seq", "cropped ref seq"], index=df.index)

    # descriptions (your existing logic)
    desc_detailed = df.apply(lambda r: describe_row(r, ref_dict, interpro_descriptions, simple=False), axis=1)
    desc_simple   = df.apply(lambda r: describe_row(r, ref_dict, interpro_descriptions, simple=True),  axis=1)

    # interpro ids only
    interpro_ids_only = df["interpro_affected"].apply(lambda v: [iid for iid, _ in safe_pairs(v)])

    out = pd.DataFrame({
        "HGNC_ID": df["HGNC_ID"].values,
        "Gene": df["GeneSymbol"].values,
        "Assembly": df["Assembly"].values,
        "Mutated Sequence": df["_mut_seq"].values,
        "Reference Sequence": df["_ref_seq"].values,
        "affected_aa_start": df["affected_aa_start"].values,
        "affected_ref_aa_end": df["affected_ref_aa_end"].values,
        "affected_mut_aa_end": df["affected_mut_aa_end"].values,
        "cropped mut seq": crop_df["cropped mut seq"].values,
        "cropped ref seq": crop_df["cropped ref seq"].values,
        "Phenotypes": df["_PhenotypesClean"].values,
        "ClinicalSignificance": df["ClinicalSignificance"].astype(str).values,
        "MutationType": df["MutationType"].astype(str).values,
        "Levenshtein": df["Levenshtein"].values,
        "interpro_ids": interpro_ids_only.values,
        "Description": desc_detailed.values,
        "DescriptionSimple": desc_simple.values,
    })
    return out

tqdm.pandas()
training_data = build_mutation_descriptions(clinvar_mut, clinvar_ref_dict, interpro_descriptions)
training_data.to_csv('/data/dandreas/MutationModel/data/training_data.csv', index=False)


100%|██████████| 114503/114503 [00:01<00:00, 105418.88it/s]


In [14]:
training_data[training_data['MutationType']=="Nonsense"]

,HGNC_ID,Assembly,Mutated Sequence,Reference Sequence,affected_aa_start,affected_ref_aa_end,affected_mut_aa_end,cropped mut seq,cropped ref seq,Phenotypes,ClinicalSignificance,MutationType,Levenshtein,interpro_ids,Description,DescriptionSimple
5,HGNC:6024,GRCh37,MTILGTTFGMVFSLLQVVSGESGYA,MTILGTTFGMVFSLLQVVSGESGYAQNGDLEDAELDDYSFSCYSQL...,25.0,459.0,25.0,MTILGTTFGMVFSLLQVVSGESGYA,MTILGTTFGMVFSLLQVVSGESGYAQNGDLEDAELDDYSFSCYSQL...,Immunodeficiency 104,Pathogenic,Nonsense,434,"[IPR003961, IPR040997, IPR013783, IPR003531]",loss-of-function (premature stop) in IL7R; Cli...,loss-of-function (premature stop) in IL7R; Cli...
12,HGNC:583,GRCh37,MAAASYDQLLKQVEALKMENSNLRQELEDNSNHLTKLETEASNMKE...,MAAASYDQLLKQVEALKMENSNLRQELEDNSNHLTKLETEASNMKE...,421.0,2843.0,421.0,MAAASYDQLLKQVEALKMENSNLRQELEDNSNHLTKLETEASNMKE...,MAAASYDQLLKQVEALKMENSNLRQELEDNSNHLTKLETEASNMKE...,Familial adenomatous polyposis 1,Pathogenic,Nonsense,2422,"[IPR026818, IPR041257, IPR009234, IPR009223, I...",loss-of-function (premature stop) in APC; Clin...,loss-of-function (premature stop) in APC; Clin...
15,HGNC:9871,GRCh37,MMAAEAGSEEGGPVTAGAGGGGAAAGSSAYPAVCRVKIPAALPVAA...,MMAAEAGSEEGGPVTAGAGGGGAAAGSSAYPAVCRVKIPAALPVAA...,318.0,1047.0,318.0,MMAAEAGSEEGGPVTAGAGGGGAAAGSSAYPAVCRVKIPAALPVAA...,MMAAEAGSEEGGPVTAGAGGGGAAAGSSAYPAVCRVKIPAALPVAA...,Capillary malformation-arteriovenous malformat...,Pathogenic,Nonsense,729,"[IPR000008, IPR011993, IPR001849, IPR001936, I...",loss-of-function (premature stop) in RASA1; Cl...,loss-of-function (premature stop) in RASA1; Cl...
22,HGNC:1058,GRCh37,MAAVPQNNLQEQLERHSARTLNNKLSLSKPKFSGFTFKKKTSSDNN...,MAAVPQNNLQEQLERHSARTLNNKLSLSKPKFSGFTFKKKTSSDNN...,271.0,1417.0,271.0,MAAVPQNNLQEQLERHSARTLNNKLSLSKPKFSGFTFKKKTSSDNN...,MAAVPQNNLQEQLERHSARTLNNKLSLSKPKFSGFTFKKKTSSDNN...,Bloom syndrome,Pathogenic,Nonsense,1146,"[IPR032284, IPR012532, IPR011545, IPR004589, I...",loss-of-function (premature stop) in BLM; Clin...,loss-of-function (premature stop) in BLM; Clin...
25,HGNC:9884,GRCh37,MPPKTPRKTAATAAAAAAEPPAPPPPPPPEEDPEQDSGPEDLPLVR...,MPPKTPRKTAATAAAAAAEPPAPPPPPPPEEDPEQDSGPEDLPLVR...,81.0,928.0,81.0,MPPKTPRKTAATAAAAAAEPPAPPPPPPPEEDPEQDSGPEDLPLVR...,MPPKTPRKTAATAAAAAAEPPAPPPPPPPEEDPEQDSGPEDLPLVR...,Retinoblastoma,Pathogenic,Nonsense,847,"[IPR013763, IPR036915, IPR028309, IPR002720, I...",loss-of-function (premature stop) in RB1; Clin...,loss-of-function (premature stop) in RB1; Clin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114478,HGNC:644,GRCh38,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,858.0,920.0,858.0,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,Androgen resistance syndrome; Kennedy disease,Pathogenic,Nonsense,62,"[IPR050200, IPR000536, IPR035500]",loss-of-function (premature stop) in AR; Clini...,loss-of-function (premature stop) in AR; Clini...
114484,HGNC:644,GRCh38,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,806.0,920.0,806.0,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,MEVQLGLGRVYPRPPSKTYRGAFQNLFQSVREVIQNPGPRHPEAAS...,Androgen resistance syndrome; Kennedy disease,Pathogenic,Nonsense,114,"[IPR050200, IPR000536, IPR035500]",loss-of-function (premature stop) in AR; Clini...,loss-of-function (premature stop) in AR; Clini...
114489,HGNC:14124,GRCh38,MRDNTSPISVILVSSGSRGNKLLFRYPFQRSQEHPASQTSKPRSRY...,MRDNTSPISVILVSSGSRGNKLLFRYPFQRSQEHPASQTSKPRSRY...,491.0,569.0,491.0,MRDNTSPISVILVSSGSRGNKLLFRYPFQRSQEHPASQTSKPRSRY...,MRDNTSPISVILVSSGSRGNKLLFRYPFQRSQEHPASQTSKPRSRY...,"Epilepsy, familial focal, with variable foci 3",Pathogenic,Nonsense,78,"[IPR056603, IPR005365]",loss-of-function (premature stop) in NPRL3; Cl...,loss-of-function (premature stop) in NPRL3; Cl...
114499,HGNC:44,GRCh38,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,59.0,686.0,59.0,MRLPDLRPWTSLLLVDAALLWLLQGPLGTLLPQGLPGLWLEGTLRL...,MRLPDLRPWTSLLLVDA

In [3]:
training_data = pd.read_csv('/data/dandreas/MutationModel/data/training_data.csv')

# #################### sequences #####################
# # save training_data['MutatedSeqence'] to a .txt file with one sequence per line
# with open('/data/dandreas/MutationModel/data/training_mutated_sequences.txt', 'w') as f:
#     for seq in training_data['Mutated Sequence']:
#         f.write(seq + '\n')

# # save unique training_data['ReferenceSequence'] to a .txt file with one sequence per line in order
# ref_seqs = training_data['Reference Sequence'].dropna().unique()
# with open('/data/dandreas/MutationModel/data/training_reference_sequences.txt', 'w') as f:
#     for seq in ref_seqs:
#         f.write(seq + '\n')

# # map ref sequences to mutated sequences
# ref_idxs = {seq: i for i, seq in enumerate(ref_seqs)}
# training_data['ReferenceIndex'] = training_data['Reference Sequence'].map(ref_idxs)

# map_mut_to_ref = {ni:i for ni, i in enumerate(list(training_data['ReferenceIndex']))}
# pickleSave(map_mut_to_ref, '/data/dandreas/MutationModel/data/', 'map_mut_to_ref.pkl')

#################### text #####################
# with open('/data/dandreas/MutationModel/data/training_descriptions.txt', 'w') as f:
#     for t in training_data['Description']:
#         f.write(t + '\n')

# with open('/data/dandreas/MutationModel/data/training_descriptions_simple.txt', 'w') as f:
#     for t in training_data['DescriptionSimple']:
#         f.write(t + '\n')

# Tokenize training dataset

In [ ]:
import numpy as np, pandas as pd
from tqdm.auto import tqdm
from transformers import AutoTokenizer

TEXT_MAX_LEN = None
SEQ_MAX_LEN  = None
BATCH_SIZE   = 512

def _to_str_list(x):
    if isinstance(x, (np.ndarray, pd.Series)): x = x.tolist()
    else: x = list(x)
    out = []
    for v in x:
        if v is None or (isinstance(v, float) and np.isnan(v)): out.append("")
        else: out.append(str(v))
    return out

def _batch_encode_ids(tokenizer, seqs, batch_size=256, desc="", max_length=None):
    """Return list[{'input_ids': list[int]}] only."""
    out = []
    n = len(seqs)
    for i in tqdm(range(0, n, batch_size), desc=desc, leave=False):
        chunk = _to_str_list(seqs[i:i+batch_size])
        enc = tokenizer(
            chunk,
            return_tensors=None,
            add_special_tokens=True,
            truncation=(max_length is not None),
            max_length=max_length,
            return_attention_mask=False,
            return_token_type_ids=False,
        )
        # Some tokenizers may still include extra keys; force-keep only input_ids
        ids_list = enc["input_ids"]
        for ids in ids_list:
            out.append({"input_ids": ids})
    return out

def _col_from_dict_list(dict_list, prefix):
    """Flatten list[{'input_ids': list[int]}] -> {f'{prefix}_input_ids': list[list[int]]}"""
    return {f"{prefix}_input_ids": [d.get("input_ids", []) for d in dict_list]}

def build_and_save_tokenized(
    training_df: pd.DataFrame,
    plm_model: str = "esm2",
    text_model: str = 'thomas-sounack/BioClinical-ModernBERT-base',
    drop_raw=True,           # drop the raw strings we tokenize
    include_crops=True       # also tokenize cropped mut/ref if present
):
    tok_df = training_df.copy()

    # Raw fields
    texts    = tok_df["Description"].fillna("").astype(str).tolist()
    texts_simple = tok_df["DescriptionSimple"].fillna("").astype(str).tolist()
    muts     = tok_df["Mutated Sequence"].fillna("").astype(str).tolist()
    refs     = tok_df["Reference Sequence"].fillna("").astype(str).tolist()
    has_mcrop = include_crops and ("cropped mut seq" in tok_df.columns)
    has_rcrop = include_crops and ("cropped ref seq" in tok_df.columns)
    mcrops   = tok_df["cropped mut seq"].fillna("").astype(str).tolist() if has_mcrop else []
    rcrops   = tok_df["cropped ref seq"].fillna("").astype(str).tolist() if has_rcrop else []

    # Tokenizers
    text_tok = AutoTokenizer.from_pretrained(text_model, use_fast=True)
    if plm_model != "esm2":
        raise ValueError(f"Only esm2 path shown here; got {plm_model}")
    seq_tok  = AutoTokenizer.from_pretrained("facebook/esm2_t30_150M_UR50D", use_fast=True)

    # Tokenize -> only input_ids
    text_tokens   = _batch_encode_ids(text_tok, texts,  BATCH_SIZE, "Text tokenizing",        TEXT_MAX_LEN)
    text_simple_tokens = _batch_encode_ids(text_tok, texts_simple, BATCH_SIZE, "Text simple tokenizing", TEXT_MAX_LEN)
    mut_tokens    = _batch_encode_ids(seq_tok,  muts,   BATCH_SIZE, "Protein (mut) tokenizing",  SEQ_MAX_LEN)
    ref_tokens    = _batch_encode_ids(seq_tok,  refs,   BATCH_SIZE, "Protein (ref) tokenizing",  SEQ_MAX_LEN)
    mcrop_tokens  = _batch_encode_ids(seq_tok,  mcrops, BATCH_SIZE, "Protein (mut-crop) tokenizing", SEQ_MAX_LEN) if has_mcrop else []
    rcrop_tokens  = _batch_encode_ids(seq_tok,  rcrops, BATCH_SIZE, "Protein (ref-crop) tokenizing", SEQ_MAX_LEN) if has_rcrop else []

    # Attach columns (keep all original columns)
    tok_df["DescriptionTokenized"] = _col_from_dict_list(text_tokens, "text")["text_input_ids"]
    tok_df["DescriptionSimpleTokenized"] = _col_from_dict_list(text_simple_tokens, "text_simple")["text_simple_input_ids"]
    tok_df["MutatedSequenceTokenized"]  = _col_from_dict_list(mut_tokens,  "mut")["mut_input_ids"]
    tok_df["ReferenceSequenceTokenized"]  = _col_from_dict_list(ref_tokens,  "ref")["ref_input_ids"]
    if has_mcrop: tok_df["MutatedSequenceTokenizedCropped"] = _col_from_dict_list(mcrop_tokens, "mutcrop")["mutcrop_input_ids"]
    if has_rcrop: tok_df["ReferenceSequenceTokenizedCropped"] = _col_from_dict_list(rcrop_tokens, "refcrop")["refcrop_input_ids"]

    # Optionally drop the raw strings we just tokenized
    if drop_raw:
        drop_cols = ["Description", "Mutated Sequence", "Reference Sequence", "DescriptionSimple"]
        if has_mcrop: drop_cols.append("cropped mut seq")
        if has_rcrop: drop_cols.append("cropped ref seq")
        tok_df.drop(columns=[c for c in drop_cols if c in tok_df.columns], inplace=True)
    
    #rename
    tok_df.rename(columns={"DescriptionTokenized": "Description"}, inplace=True)
    tok_df.rename(columns={"DescriptionSimpleTokenized": "DescriptionSimple"}, inplace=True)
    tok_df.rename(columns={"MutatedSequenceTokenized": "MutatedSequence"}, inplace=True)
    tok_df.rename(columns={"ReferenceSequenceTokenized": "ReferenceSequence"}, inplace=True)
    tok_df.rename(columns={"MutatedSequenceTokenizedCropped": "MutatedSequenceCropped"}, inplace=True)
    tok_df.rename(columns={"ReferenceSequenceTokenizedCropped": "ReferenceSequenceCropped"}, inplace=True)

    return tok_df


# 7) save
tokenized_df = build_and_save_tokenized(
    training_df=training_data,
    drop_raw=True,            # drop raw string cols that are tokenized
    include_crops=True        # tokenize cropped sequences if present
)

tokenized_df.to_pickle('/data/dandreas/MutationModel/data/training_data_tokenized.pkl') 



In [16]:
tokenized_df

,HGNC_ID,Assembly,affected_aa_start,affected_ref_aa_end,affected_mut_aa_end,Phenotypes,ClinicalSignificance,MutationType,Levenshtein,interpro_ids,Description,DescriptionSimple,MutatedSequence,ReferenceSequence,MutatedSequenceCropped,ReferenceSequenceCropped
0,HGNC:6597,GRCh37,409.0,1097.0,422.0,Stüve-Wiedemann syndrome 1,Pathogenic,Frame_Shift_Deletion,688,"[IPR003961, IPR013783, IPR003529, IPR050379]","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[0, 20, 20, 13, 12, 19, 7, 23, 4, 15, 10, 14, ...","[0, 20, 20, 13, 12, 19, 7, 23, 4, 15, 10, 14, ...","[0, 20, 20, 13, 12, 19, 7, 23, 4, 15, 10, 14, ...","[0, 20, 20, 13, 12, 19, 7, 23, 4, 15, 10, 14, ..."
1,HGNC:6597,GRCh37,540.0,1097.0,547.0,Stüve-Wiedemann syndrome 1,Pathogenic,Frame_Shift_Insertion,557,"[IPR003961, IPR013783, IPR003529, IPR050379]","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[0, 20, 20, 13, 12, 19, 7, 23, 4, 15, 10, 14, ...","[0, 20, 20, 13, 12, 19, 7, 23, 4, 15, 10, 14, ...","[0, 20, 20, 13, 12, 19, 7, 23, 4, 15, 10, 14, ...","[0, 20, 20, 13, 12, 19, 7, 23, 4, 15, 10, 14, ..."
2,HGNC:6024,GRCh37,196.0,459.0,239.0,Immunodeficiency 104,Pathogenic,Frame_Shift_Deletion,263,"[IPR003961, IPR013783, IPR003531]","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18..."
3,HGNC:6024,GRCh37,164.0,459.0,166.0,Immunodeficiency 104,Pathogenic,Frame_Shift_Deletion,295,"[IPR003961, IPR013783, IPR003531]","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18..."
4,HGNC:6024,GRCh37,58.0,459.0,79.0,Immunodeficiency 104,Pathogenic,Frame_Shift_Deletion,401,"[IPR003961, IPR040997, IPR013783, IPR003531]","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[50281, 18585, 14, 1171, 14, 3701, 313, 27388,...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18...","[0, 20, 11, 12, 4, 6, 11, 11, 18, 6, 20, 7, 18..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114498,HGNC:4932,GRCh38,122.0,123.0,123.0,,Likely benign,Substitution,1,"[IPR050208, IPR001039, IPR011161, IPR011162]","[50281, 2377, 5379, 313, 6795, 28, 337, 39951,...","[50281, 2377, 5379, 313, 6795, 28, 337, 39951,...","[0, 20, 4, 7, 20, 5, 14, 10, 11, 7, 4, 4, 4, 4...","[0, 20, 4, 7, 20, 5, 14, 10, 11, 7, 4, 4, 4, 4...","[0, 20, 4, 7, 20, 5, 14, 10, 11, 7, 4, 4, 4, 4...","[0, 20, 4, 7, 20, 5, 14, 10, 11, 7, 4, 4, 4, 4..."
114499,HGNC:44,GRCh38,59.0,686.0,59.0,MHC class I deficiency,Pathogenic,Nonsense,627,"[IPR003593, IPR013305, IPR011527, IPR036640, I...","[50281, 18585, 14, 1171, 14, 3701, 313, 37448,...","[50281, 18585, 14, 1171, 14, 3701, 313, 37448,...","[0, 20, 10, 4, 14, 13, 4, 10, 14, 22, 11, 8, 4...","[0, 20, 10, 4, 14, 13, 4, 10, 14, 22, 11, 8, 4...","[0, 20, 10, 4, 14, 13, 4, 10, 14, 22, 11, 8, 4...","[0, 20, 10, 4, 14, 13, 4, 10, 14, 22, 11, 8, 4..."
114500,HGNC:4170,GRCh38,52.0,53.0,53.0,GATA binding protein 1 related thrombocytopeni...,Benign,Substitution,1,[IPR039355],"[50281, 2377, 5379, 313, 6795, 28, 337, 39951,...","[50281, 2377, 5379, 313, 6795, 28, 337, 39951,...","[0, 20, 9, 18, 14, 6, 4, 6, 8, 4, 6, 11, 8, 9,...","[0, 20, 9, 18, 14, 6, 4, 6, 8, 4, 6, 11, 8, 9,...","[0, 20, 9, 18, 14, 6, 4, 6, 8, 4, 6, 11, 8, 9,...","[0, 20, 9, 18, 14, 6, 4, 6, 8, 4, 6, 11, 8, 9,..."
114501,HGNC:10295,GRCh38,828.0,1020.0,832.0,Retinitis pigmentosa 3; Primary ciliary dyskin...,Pathogeni

# Embeddings only

In [ ]:
####### SUM EMBEDDINGS ########

import pandas as pd
import os
import numpy as np
from saveAndLoad import *
from tqdm import tqdm
import h5py

mut_emb = h5py.File(f'/data/dandreas/MutationModel/data/training_mutated_embeddings.h5', 'r')['embeddings']
ref_emb = h5py.File(f'/data/dandreas/MutationModel/data/training_reference_embeddings.h5', 'r')['embeddings']
out_path_ref = f'/data/dandreas/MutationModel/data/training_reference_sum_embeddings.h5'
out_path_mut = f'/data/dandreas/MutationModel/data/training_mutated_sum_embeddings.h5'

def get_sum_embeddings(emb, out_path, emb_dim = 640):
    if os.path.exists(out_path):
        print(f'\tremoved {out_path}')
        os.remove(out_path)

    h5f_file = h5py.File(out_path, 'a')
    embeddings = h5f_file.create_dataset(
        'embeddings',
        shape=(len(emb),emb_dim),
        dtype=np.dtype('float32'),
        chunks=(1,emb_dim)
    )

    print(f'\t\t{out_path} created with shape {embeddings.shape}')

    sum_embedding = lambda emb: np.zeros(emb_dim, dtype=np.float32) if (emb.size == 0) else emb[:].sum(axis=0)
    for ni,i in tqdm(enumerate(emb),total=len(emb)):
        embedding = i.reshape(-1,emb_dim)
        embedding = sum_embedding(embedding)
        embeddings[ni] = embedding

    h5f_file.close()

get_sum_embeddings(mut_emb, out_path_mut)
get_sum_embeddings(ref_emb, out_path_ref)

In [48]:
########################## DIFF EMBEDDINGS ############################
mut2ref = pickleLoad('/data/dandreas/MutationModel/data/map_mut_to_ref.pkl')
mut_emb_sum = h5py.File(f'/data/dandreas/MutationModel/data/training_mutated_sum_embeddings.h5', 'r')['embeddings']
ref_emb_sum = h5py.File(f'/data/dandreas/MutationModel/data/training_reference_sum_embeddings.h5', 'r')['embeddings']
with open('/data/dandreas/MutationModel/data/training_mutated_sequences.txt', 'r') as f:
    mut_seqs = f.readlines()
with open('/data/dandreas/MutationModel/data/training_reference_sequences.txt', 'r') as f:
    ref_seqs = f.readlines()

def analyze(mode, mut_emb, ref_emb, mut_seqs, ref_seqs, mut2ref, out_path, emb_dim = 640):

    print(f'mode:{mode}\nmut_emb[0].shape: {mut_emb[0].shape}\nOUTPUT PATH: {out_path}')

    if os.path.exists(out_path):
        print(f'\tremoved {out_path}')
        os.remove(out_path)

    #CREATE OUTPUT HDF5 FILE
    h5f = h5py.File(out_path, 'a')
    embeddings = h5f.create_dataset(
        'embeddings',
        shape=(len(mut_emb),emb_dim),
        dtype=np.dtype('float32'),
        chunks=(1,emb_dim)
    )
    print(f'\t\t{out_path} created with shape {embeddings.shape}')

    #ANALYZE
    for mut_idx, mut_embedding in tqdm(enumerate(mut_emb), total=len(mut_emb)):
        ref_idx = mut2ref[mut_idx]
        ref_embedding = ref_emb[ref_idx]
        if mode == 'mean':
            mut_embedding = mut_embedding / len(mut_seqs[mut_idx].strip())  # assuming mut_seqs is a list of sequences
            ref_embedding = ref_embedding / len(ref_seqs[ref_idx].strip())  # assuming ref_seqs is a list of sequences
        elif mode == 'scaled_sum':
            mut_embedding = mut_embedding * len(ref_seqs[ref_idx].strip()) / len(mut_seqs[mut_idx].strip())
        elif mode != 'sum':
            raise ValueError(f"Unknown mode: {mode}")
            
        embeddings[mut_idx] = mut_embedding - ref_embedding

    h5f.close()

for m in ['mean', 'sum', 'scaled_sum']:
    analyze(m, mut_emb_sum, ref_emb_sum, mut_seqs, ref_seqs, mut2ref, out_path=f'/data/dandreas/MutationModel/data/training_diff_embeddings_{m}.h5', emb_dim=640)

loading data from /data/dandreas/MutationModel/data/map_mut_to_ref.pkl
mode:mean
mut_emb[0].shape: (640,)
OUTPUT PATH: /data/dandreas/MutationModel/data/training_diff_embeddings_mean.h5
	removed /data/dandreas/MutationModel/data/training_diff_embeddings_mean.h5
		/data/dandreas/MutationModel/data/training_diff_embeddings_mean.h5 created with shape (114503, 640)


100%|██████████| 114503/114503 [00:06<00:00, 17169.53it/s]


mode:sum
mut_emb[0].shape: (640,)
OUTPUT PATH: /data/dandreas/MutationModel/data/training_diff_embeddings_sum.h5
		/data/dandreas/MutationModel/data/training_diff_embeddings_sum.h5 created with shape (114503, 640)


100%|██████████| 114503/114503 [00:05<00:00, 19763.34it/s]


mode:scaled_sum
mut_emb[0].shape: (640,)
OUTPUT PATH: /data/dandreas/MutationModel/data/training_diff_embeddings_scaled_sum.h5
		/data/dandreas/MutationModel/data/training_diff_embeddings_scaled_sum.h5 created with shape (114503, 640)


100%|██████████| 114503/114503 [00:06<00:00, 16911.32it/s]


In [ ]:
###### TEXT CLS EMBEDDINGS ######

def get_cls_embeddings(descriptions, out_path, emb_dim=768):
    if os.path.exists(out_path):
        print(f'\tremoved {out_path}')
        os.remove(out_path)
    h5f = h5py.File(out_path, 'a')
    embeddings = h5f.create_dataset(
        'embeddings',
        shape=(len(descriptions), emb_dim),
        dtype=np.dtype('float32'),
        chunks=(1, emb_dim)
    )
    print(f'\t\t{out_path} created with shape {embeddings.shape}')

    for ni,i in tqdm(enumerate(descriptions), total=len(descriptions)):
        embedding = i.reshape(-1, emb_dim)
        cls = embedding[0]
        embeddings[ni] = cls
    h5f.close()

emb_dim = 768
descriptions = h5py.File(f'/data/dandreas/MutationModel/data/training_descriptions_embeddings.h5', 'r')['embeddings']
out_path = f'/data/dandreas/MutationModel/data/training_descriptions_cls_embeddings.h5'
get_cls_embeddings(descriptions, out_path, emb_dim=emb_dim)

descriptions_simple = h5py.File(f'/data/dandreas/MutationModel/data/training_descriptions_simple_embeddings.h5', 'r')['embeddings']
out_path_simple = f'/data/dandreas/MutationModel/data/training_descriptions_simple_cls_embeddings.h5'
get_cls_embeddings(descriptions_simple, out_path_simple, emb_dim=emb_dim)


	removed /data/dandreas/MutationModel/data/training_descriptions_cls_embeddings.h5
		/data/dandreas/MutationModel/data/training_descriptions_cls_embeddings.h5 created with shape (114503, 768)


100%|██████████| 114503/114503 [06:34<00:00, 290.24it/s]


		/data/dandreas/MutationModel/data/training_descriptions_simple_cls_embeddings.h5 created with shape (114503, 768)


100%|██████████| 114503/114503 [00:19<00:00, 5745.60it/s]


In [73]:
clinvar_mut.columns

Index(['HGNC_ID', 'Assembly', 'Chromosome', 'Start', 'Stop', 'Type',
       'ReferenceAlleleVCF', 'AlternateAlleleVCF', 'Sequence', 'Name',
       'uniprot_id', 'mut_start', 'mut_end', 'interpro_overlap', 'AlleleID',
       'GeneID', 'GeneSymbol', 'ClinicalSignificance', 'ClinSigSimple',
       'LastEvaluated', 'RS# (dbSNP)', 'nsv/esv (dbVar)', 'RCVaccession',
       'PhenotypeIDS', 'PhenotypeList', 'Origin', 'OriginSimple',
       'ChromosomeAccession', 'ReferenceAllele', 'AlternateAllele',
       'Cytogenetic', 'ReviewStatus', 'NumberSubmitters', 'Guidelines',
       'TestedInGTR', 'OtherIDs', 'SubmitterCategories', 'VariationID',
       'PositionVCF', 'SomaticClinicalImpact',
       'SomaticClinicalImpactLastEvaluated', 'ReviewStatusClinicalImpact',
       'Oncogenicity', 'OncogenicityLastEvaluated', 'ReviewStatusOncogenicity',
       'SCVsForAggregateGermlineClassification',
       'SCVsForAggregateSomaticClinicalImpact',
       'SCVsForAggregateOncogenicityClassification', 'Mutati

In [72]:
training_data.to_csv('/data/dandreas/MutationModel/data/training_data.csv', index=False)

In [ ]:
x ='''
loss-of-function (frameshift) in LIFR; ClinSig: Pathogenic; Phenotypes: Stüve-Wiedemann syndrome 1; Fibronectin type III: Fibronectin is a dimeric glycoprotein composed of disulfide-linked subunits with a molecular weight of 220-250kDa each. It is involved in cell adhesion, cell morphology, thrombosis, cell migration, and embryonic differentiation. Fibronectin is a modular protein composed of homologous repeats of three prototypical types of domains known as types I, II, and III. Fibronectin type-III (FN3) repeats are both the largest and the most common of the fibronectin subdomains. Domains homologous to FN3 repeats have been found in various animal protein families including other extracellular-matrix molecules, cell-surface receptors, enzymes, and muscle proteins. Structures of individual FN3 domains have revealed a conserved β-sandwich fold with one β-sheet containing four strands and the other sheet containing three strands (see for example ). This fold is topologically very similar to that of Ig-like domains, with a notable difference being the lack of a conserved disulfide bond in FN3 domains. Distinctive hydrophobic core packing and the lack of detectable sequence homology between immunoglobulin and FN3 domains suggest, however, that these domains are not evolutionarily related. FN3 exhibits functional as well as structural modularity. Sites of interaction with other molecules have been mapped to short stretch of amino acids such as the Arg-Gly-Asp (RGD) sequence found in various FN3 domains. The RGD sequences is involved in interactions with integrin. Small peptides containing the RGD sequence can modulate a variety of cell adhesion invents associated with thrombosis, inflammation, and tumour metastasis. These properties have led to the investigation of RGD peptides and RGD peptide analogues as potential therapeutic agents.; Immunoglobulin-like fold: This superfamily represents domains with an immunoglobulin-like (Ig-like) fold, which consists of a β-sandwich of seven or more strands in two sheets with a Greek-key topology. Ig-like domains are one of the most common protein modules found in animals, occurring in a variety of different proteins. These domains are often involved in interactions, commonly with other Ig-like domains via their β-sheets. Domains within this fold-family share the same structure, but can diverge with respect to their sequence. Based on sequence, Ig-like domains can be classified as V-set domains (antibody variable domain-like), C1-set domains (antibody constant domain-like), C2-set domains, and I-set domains (antibody intermediate domain-like). Proteins can contain more than one of these types of Ig-like domains. For example, in the human T-cell receptor antigen CD2, domain 1 (D1) is a V-set domain, while domain 2 (D2) is a C2-set domain, both domains having the same Ig-like fold. Domains with an Ig-like fold can be found in many, diverse proteins in addition to immunoglobulin molecules. For example, Ig-like domains occur in several different types of receptors (such as various T-cell antigen receptors), several cell adhesion molecules, MHC class I and II antigens, as well as the hemolymph protein hemolin, and the muscle proteins titin, telokin and twitchin.; Long hematopoietin receptor, Gp130 family 2, conserved site: A number of receptors for lymphokines, hematopoietic growth factors and growth hormone-related molecules have been found to share a common binding domain. These receptors are designated as hematopoietin receptors and the corresponding ligands as hematopoietins. Further, hematopoietins have been subdivided into two major structural groups: Large/long and small/short hematopoietins. One subset of individual receptor chains that are part of receptor complexes for large hematopoietins contain common structural elements in their extracellular parts: an immunoglobulin-like domain, an hematopoietin-receptor domain, and 3 fibronectin type-III domains (2 in the leptin receptor). This subgroup was designated as "Gp130 family of receptors" and contains the following chains: Leptin receptor (LPTR), Granulocyte colony stimulating factor receptor (GCSFR), Interleukin-6/-11/LIF/OSM/CNTF common beta chain (Gp130), Leukemia inhibiting factor receptor (LIFR), Oncostatin-M receptor beta chain (OSMR), Interleukin-12 receptor beta-1 chain (IL12RB1), Interleukin-12 receptor beta-2 chain (IL12RB2). A schematic representation of the structure of these receptors is shown below: These receptor chains homodimerize (GCSFR, Gp130, LPTR) or heterodimerize (Gp130 with LIFR or OSMR, IL12RB1 with IL12RB2) upon binding of the cognate cytokine: G-CSF, LIF, OSM, LPT, or the cytokine/alpha chain complex: IL-6/IL6RA, IL-11/IL11RA, CNTF/CNTFRA, IL-12 (p35/p40).; Type I Cytokine Receptor: The Type I Cytokine Receptor family is characterized by its role in immune system regulation and hematopoiesis. Members of this family are involved in various signaling pathways, such as JAK/STAT, MAPK, and NOTCH, which lead to diverse biological effects including cell proliferation, differentiation, survival, and inflammation control. They are receptors for a range of cytokines, including interleukins, growth hormone, prolactin, and leptin, mediating responses like growth regulation, appetite control, and immune cell development. The receptors typically feature a WSXWS motif essential for proper folding and transport, and a box 1 motif necessary for interaction with JAK kinases. Some receptors also exist in soluble forms, acting as agonists or modulators of signaling. This family plays a critical role in both central and peripheral functions, affecting processes from bone metabolism to neural development and regeneration.
'''

279


In [9]:
list(clinvar_ref_dict.items())[0]

(('HGNC:22197', 'GRCh37'),
 {'Sequence': 'MFSAGAESLLHQAREIQDEELKKFCSRICKLLQAEDLGPDTLDSLQRLFLIISATKYSRRLEKTCVDLLQATLGLPACPEQLQVLCAAILREMSPSDSLSLAWDHTQNSRQLSLVASVLLAQGDRNEEVRAVGQGVLRALESRQPEGPSLRHLLPVMAKVVVLSPGTLQEDQATLLSKRLVDWLRYASLQQGLPHSGGFFSTPRARQPGPVTEVDGAVATDFFTVLSSGHRFTDDQWLNVQAFSMLRAWLLHSGPEGPGTLDTDDRSEQEGSTLSVISATSSAGRLLPPRERLREVAFEYCQRLIEQSNRRALRKGDSDLQKACLVEAVLVLDVLCRQDPSFLYRSLSCLKALHGRVRGDPASVRVLLPLAHFFLSHGEAAAVDSEAVYQHLFTRIPVEQFHSPMLAFEFIQFCRDNLHLFSGHLSTLRLSFPNLFKFLAWNSPPLTSEFVALLPALVDAGTALEMLHALLDLPCLTAVLDLQLRSAPAASERPLWDTSLRAPSCLEAFRDPQFQGLFQYLLRPKASGATERLAPLHQLLQPMAGCARVAQCAQAVPTLLQAFFSAVTQVADGSLINQLALLLLGRSDSLYPAPGYAAGVHSVLSSQFLALCTLKPSLVVELARDLLEFLGSVNGLCSRASLVTSVVWAIGEYLSVTYDRRCTVEQINKFFEALEALLFEVTQCRPSAALPRCPPQVVTVLMTTLTKLASRSQDLIPRASLLLSKMRTLAHSPATSSTHSEEGAEAIRTRATELLTLLKMPSVAQFVLTPSTEVCSPRYHRDANTALPLALRTVSRLVEREAGLMPG'})

In [11]:
list(interpro_descriptions.items())[0]

('IPR000001',
 'Kringles are autonomous structural domains, found throughout the blood clotting and fibrinolytic proteins. Kringle domains are believed to play a role in binding mediators (e.g., membranes, other proteins or phospholipids), and in the regulation of proteolytic activity. Kringle domains are characterised by a triple loop, 3-disulphide bridge structure, whose conformation is defined by a number of hydrogen bonds and small pieces of anti-parallel β-sheet. They are found in a varying number of copies in some plasma proteins including prothrombin and urokinase-type plasminogen activator, which are serine proteases belonging to MEROPS peptidase family S1A.')

In [14]:
protein2ipr_human.head()

,uniprot_id,interpro_id,description,source_id,start,end
0,A0A024R161,IPR001623,DnaJ domain,PF00226,49,115
1,A0A024R161,IPR001623,DnaJ domain,PR00625,51,69
2,A0A024R161,IPR001623,DnaJ domain,PR00625,69,84
3,A0A024R161,IPR001623,DnaJ domain,PR00625,96,116
4,A0A024R161,IPR001623,DnaJ domain,PS50076,49,124


In [15]:
list(hgnc_uniprot_name_map.items())[0]

('HGNC:12849', 'P31946')

# Clean clinvar data (remove duplicates in GRCh37 and 38)

In [ ]:
clinvar_ref_path = '/data/dandreas/clinvar/ref_sequences.csv'
clinvar_mut_path = '/data/dandreas/clinvar/mut_sequences.csv'

clinvar_ref = pd.read_csv(clinvar_ref_path)
clinvar_mut = pd.read_csv(clinvar_mut_path)

# ── for clinvar_ref ──────────────────────────────────────────────
key_ref = ["HGNC_ID", "Sequence"]                  # no Assembly here

# ── for clinvar_mut (example) ───────────────────────────────────
key_mut = [
    "HGNC_ID", "Chromosome", "Sequence"
]

def split_by_assembly(df, key_cols):
    tmp = (
        df.assign(flag=1)                        # mark presence
          .pivot_table(
              index=key_cols,                   # one row per unique key
              columns="Assembly",               # GRCh37 / GRCh38
              values="flag",
              fill_value=0,                     # 0 = absent
          )
    )

    both        = tmp.query("GRCh37 == 1 & GRCh38 == 1").reset_index()
    only_37     = tmp.query("GRCh37 == 1 & GRCh38 == 0").reset_index()
    only_38     = tmp.query("GRCh38 == 1 & GRCh37 == 0").reset_index()
    return both, only_37, only_38

both_ref, ref_37_only, ref_38_only = split_by_assembly(clinvar_ref, key_ref)
both_mut, mut_37_only, mut_38_only = split_by_assembly(clinvar_mut, key_mut)

print(len(ref_37_only), "unique to GRCh37 in clinvar_ref")
print(len(ref_38_only), "unique to GRCh38 in clinvar_ref")
print(len(both_ref),    "present in both assemblies")
print('')
print(len(mut_37_only), "unique to GRCh37 in clinvar_mut")
print(len(mut_38_only), "unique to GRCh38 in clinvar_mut")  
print(len(both_mut),    "present in both assemblies")


716 unique to GRCh37 in clinvar_ref
955 unique to GRCh38 in clinvar_ref
17090 present in both assemblies

128538 unique to GRCh37 in clinvar_mut
152929 unique to GRCh38 in clinvar_mut
1911820 present in both assemblies


In [ ]:
wide = (
    clinvar_ref                 # HGNC_ID | Assembly | Sequence
      .drop_duplicates(["HGNC_ID", "Assembly", "Sequence"])  # safety
      .pivot_table(index="HGNC_ID",
                   columns="Assembly",
                   values="Sequence",
                   aggfunc="first")     # rows: genes; cols: GRCh37 / GRCh38
)

# genes whose sequences differ (or are missing in one build)
mismatch = wide[wide["GRCh37"] != wide["GRCh38"]]

print(f"{len(mismatch)} genes have different sequences between assemblies")
mismatch.dropna(inplace=True)
mismatch


1094 genes have different sequences between assemblies


/tmp/ipykernel_1125771/1529366120.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mismatch.dropna(inplace=True)


Assembly,GRCh37,GRCh38
HGNC_ID,,
HGNC:10295,MREPEELMPDSGAVFTFGKSKFAENNPGKFWFKNDVPVHLSCGDEH...,MREPEELMPDSGAVFTFGKSKFAENNPGKFWFKNDVPVHLSCGDEH...
HGNC:10298,MGRRPARCYRYCKNKPYPKSRFCRGVPDAKIRIFDLGRKKAKVDEF...,MGRRPARCYRYCKNKPYPKSRFCRGVPDAKIRIFDLGRKKAKVDEF...
HGNC:10448,MDIYDTQTLGVVVFGGFMVVSAIGIFLVSTFSMKETSYEEALANQR...,MDIYDTQTLGVVVFGGFMVVSAIGIFLVSTFSMKETSYEEALANQR...
HGNC:10526,MSRRKQRKPQQLISDCEGPSASENGDASEEDHPQVCAKCCAQFTDP...,MSRRKQRKPQQLISDCEGPSASENGDASEEDHPQVCAKCCAQFTDP...
HGNC:10590,MAQALLVPPGPESFRLFTRESLAAIEKRAAEEKAKKPKKEQDNDDE...,MAQALLVPPGPESFRLFTRESLAAIEKRAAEEKAKKPKKEQDNDDE...
...,...,...
HGNC:9679,IGTSETQVDVSNVVPGTRYDITISSISTTYTSPVTRIVTTNVTKPG...,IGTSETQVDVSNVVPGTRYDITISSISTTYTSPVTRIVTTNVTKPG...
HGNC:9705,MARAMAAAWPLLLVALLVLSWPPPGTGDVVVQAPTQVPGFLGDSVT...,MARAMAAAWPLLLVALLVLSWPPPGTGDVVVQAPTQVPGFLGDSVT...
HGNC:976,MKDCSNGCSAECTGEGGSKEVVGTFKAKDLIVTPATILKEKPDPNN...,VRDCSNGCSAECTGEGGSKEVVGTFKAKDLIVTPATILKEKPDPNN...


In [ ]:
############################################################
# Helper: prefer GRCh37 over GRCh38 when duplicates found
############################################################
def dedup_keep_37(df, group_cols):
    order = pd.CategoricalDtype(['GRCh37', 'GRCh38'], ordered=True)
    return (df
            .assign(_ord=df['Assembly'].astype(order))
            .sort_values('_ord')                    # GRCh37 first
            .drop_duplicates(group_cols, keep='first')
            .drop(columns='_ord')
            .dropna(subset="Sequence")
           )

########################################################################
# 1) Consolidate clinvar_ref  – drop GRCh38 when seq is identical
########################################################################
ref_group = ['HGNC_ID', 'Sequence']       # identical seq across builds
clinvar_ref_clean = dedup_keep_37(clinvar_ref, ref_group)

print(f"clinvar_ref: {len(clinvar_ref)} → {len(clinvar_ref_clean)} rows")

########################################################################
# 2) Build whitelist of valid (HGNC_ID, Assembly) pairs
########################################################################
valid_keys = (
    clinvar_ref_clean[['HGNC_ID', 'Assembly']]
      .drop_duplicates()
      .assign(_ok=True)
)

########################################################################
# 3) Consolidate clinvar_mut
#    - drop rows whose (HGNC_ID, Assembly) no longer exists
#    - if both builds exist for same (HGNC_ID, Name),
#      keep the GRCh37 one
########################################################################
# a) remove rows whose reference build disappeared
keep_mask = (
    clinvar_mut.merge(valid_keys, on=['HGNC_ID', 'Assembly'], how='left')['_ok']
               .fillna(False)
)
mut_kept = clinvar_mut.loc[keep_mask].copy()

# b) within remaining rows, deduplicate so that
#      for each (HGNC_ID, Name) pair GRCh37 is preferred
mut_group = ['HGNC_ID', 'Name']
clinvar_mut_clean = dedup_keep_37(mut_kept, mut_group)

print(f"clinvar_mut: {len(clinvar_mut)} → {len(clinvar_mut_clean)} rows")

############################################################
# 4) Optional: sanity‑check that every clinvar_mut row
#    still finds exactly one reference row
############################################################
ref_lookup = clinvar_ref_clean.set_index(['HGNC_ID', 'Assembly'])
test = clinvar_mut_clean.set_index(['HGNC_ID', 'Assembly']).index \
        .isin(ref_lookup.index)
assert test.all(), "Some mutation rows lost their reference!"

clinvar_ref_clean.to_csv('/data/dandreas/clinvar/ref_sequences_clean.csv', index=False)
clinvar_mut_clean.to_csv('/data/dandreas/clinvar/mut_sequences_clean.csv', index=False)



clinvar_ref: 35851 → 18761 rows


/tmp/ipykernel_3099062/638233226.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(False)


clinvar_mut: 4143931 → 1996532 rows


In [ ]:
##############################################################################
# Merge clinvar_mut_clean with clinvar_ref_clean
#   – link on (HGNC_ID, Assembly)                ← the same key we used earlier
#   – keep “Name” from the mutation table
#   – rename the two Sequence columns so they’re explicit
##############################################################################

mut_vs_ref = (
    clinvar_mut_clean
      .merge(
          clinvar_ref_clean[["HGNC_ID", "Assembly", "Sequence"]],
          on=["HGNC_ID", "Assembly"],
          how="left",               # every mutation row *must* find one ref
          validate="m:1",           # many‑to‑one check: 1 ref per key
          suffixes=("_mut", "_ref") # gives Sequence_mut / Sequence_ref
      )
      .rename(
          columns={
              "Sequence_ref": "Ref_Sequence",
              "Sequence_mut": "Mut_Sequence"
          }
      )
      [["Name", "Assembly", "Ref_Sequence", "Mut_Sequence"]]   # desired order
      .reset_index(drop=True)
)

################################################################################
# Quick sanity‑check (optional but recommended)
################################################################################
assert mut_vs_ref["Ref_Sequence"].notna().all(), "Some rows lost their reference!"
assert mut_vs_ref["Mut_Sequence"].notna().all(), "Mutation sequence missing!"

mut_vs_ref.to_csv('/data/dandreas/clinvar/mutated_seqs_clean.csv', index=False)


# InterPro

In [ ]:
# interpro_descriptions_df = pd.read_json('/data/dandreas/MutationModel/data/interpro_descriptions.json')
# interpro_descriptions_df = interpro_descriptions_df.set_index('interpro_id')

# def clean_placeholders(text: str) -> str:
#     # 1) the two placeholder patterns – no word-chars allowed inside
#     EMPTY_BRACKETS = re.compile(r'\s*\[[^\w]*\]')   #  [… ] with nothing but spaces / punctuation
#     EMPTY_PARENS   = re.compile(r'\s*\([^\w]*\)')   #  (… ) idem
#     # 2) generic whitespace tidy-ups
#     SPACES       = re.compile(r'\s+')
#     BEFORE_PUNCT = re.compile(r'\s+([.,;:?!])')
#     text = EMPTY_BRACKETS.sub('', text)   # e.g. “[ , , ]”  →  ''
#     text = EMPTY_PARENS.sub('', text)     # e.g. “( )”      →  ''
#     text = SPACES.sub(' ', text)          # collapse runs of spaces / new-lines
#     text = BEFORE_PUNCT.sub(r'\1', text)  # strip any space that sneaks in before punctuation
#     return text.strip()


# interpro_dict = {}
# for ni,i in tqdm(enumerate(interpro_descriptions_df.itertuples()), total = len(interpro_descriptions_df)):
#     interpro_dict[i.Index] = clean_placeholders(i.description)

# pickleSave(interpro_dict, '/data/dandreas/MutationModel/data/', 'interpro_descriptions.pkl')

100%|██████████| 48469/48469 [00:02<00:00, 21463.54it/s]


saved to /data/dandreas/MutationModel/data/interpro_descriptions.pkl


In [ ]:
clinvar_ref_path = '/data/dandreas/clinvar/ref_sequences_clean.csv'
clinvar_mut_path = '/data/dandreas/clinvar/mut_sequences_clean.csv'

clinvar_ref = pd.read_csv(clinvar_ref_path)
clinvar_mut = pd.read_csv(clinvar_mut_path)
interpro_descriptions = pickleLoad('/data/dandreas/MutationModel/data/interpro_descriptions.pkl')
protein2ipr_human = pd.read_csv('./data/protein2ipr_human.tsv', sep='\t')
hgnc_uniprot_name_map = pickleLoad('/home/dandreas/SomaticMutationLLM/hgnc_names/hgnc_uniprot_name_map.pkl')
clinvar_ref_dict = clinvar_ref.set_index(['HGNC_ID', 'Assembly']).T.to_dict()

loading data from /data/dandreas/MutationModel/data/interpro_descriptions.pkl
loading data from /home/dandreas/SomaticMutationLLM/hgnc_names/hgnc_uniprot_name_map.pkl


In [ ]:
# maxlen = ''
# for k,i in interpro_descriptions.items():
#     if len(i) > len(maxlen):
#         maxlen = i
# print(f'Max length of interpro description: {len(maxlen.split(" "))} words\n{k}: {maxlen}')

loading data from /data/dandreas/MutationModel/data/interpro_descriptions.pkl
Max length of interpro description: 1195 words
IPR058129: This group of cysteine peptidases belong to the MEROPS peptidase family C2 (calpain family, clan CA). A type example is calpain, which is an intracellular protease involved in many important cellular functions that are regulated by calcium. The protein is a complex of 2 polypeptide chains (light and heavy), with eleven known active peptidases in humans and two non-peptidase homologues known as calpamodulin and androglobin. These include a highly calcium-sensitive (i.e., micro-molar range) form known as mu-calpain, mu-CANP or calpain I; a form sensitive to calcium in the milli-molar range, known as m-calpain, m-CANP or calpain II; and a third form, known as p94, which is found in skeletal muscle only. All forms have identical light but different heavy chains. Both mu- and m-calpain are heterodimers containing an identical 28kDa subunit and an 80kDa subu

In [4]:
# hgnc_uniprot_name_map = pickleLoad('/home/dandreas/SomaticMutationLLM/hgnc_names/hgnc_uniprot_name_map.pkl')
# human_uniprot_ids = set(hgnc_uniprot_name_map.values())
# protein2ipr_human = []
# chunk_size = 2_000_000
# ipr_path = '../data/protein2ipr.dat'
# n_rows = int(subprocess.check_output(['wc', '-l', ipr_path]).split()[0])
# n_chunks = math.ceil(n_rows / chunk_size)
# for chunk in tqdm(pd.read_csv(ipr_path, sep='\t', header = None, chunksize=chunk_size), total=n_chunks):
#     human = chunk[chunk[0].isin(human_uniprot_ids)]
#     if not human.empty:
#         protein2ipr_human.append(human)
# protein2ipr_human = pd.concat(protein2ipr_human, ignore_index=True)
# protein2ipr_human.columns = ['uniprot_id','interpro_id', 'description', 'source_id', 'start', 'end']
# protein2ipr_human.to_csv('../data/protein2ipr_human.tsv', sep='\t', index=False)

In [8]:
# HOW MANY INTERPRO DESCRIPTIONS ARE MISSING?

# in_ = 0
# not_in = 0
# for i in protein2ipr_human.interpro_id:
#     if i not in interpro_descriptions:
#         not_in += 1
#     else:
#         in_ += 1
# print(f'In:\t\t{in_}\t\tNot in:\t\t\t{not_in}')

# in_ = 0
# not_in = 0
# for i in protein2ipr_human.interpro_id.unique():
#     if i not in interpro_descriptions:
#         not_in += 1
#     else:
#         in_ += 1
# print(f'In (unique):\t{in_}\t\tNot in (unique):\t{not_in}')

# '''OUTPUT:
# In:		    238201		Not in:			    2813
# In (unique):	19080		Not in (unique):	100
# '''

In [ ]:
# # ADD INTERPRO OVERLAP LIST TO CLINVAR MUTATIONS 

# tqdm.pandas()

# ipr_by_uniprot = {
#     uid: g[['interpro_id', 'description', 'start', 'end']].to_numpy()
#     for uid, g in protein2ipr_human.groupby('uniprot_id', sort=False)
# }

# # add uniprot + [mut_start, mut_end] columns ---------------------------------
# clinvar_mut['uniprot_id'] = clinvar_mut['HGNC_ID'].map(hgnc_uniprot_name_map)

# def _mut_span(s):
#     ref = clinvar_ref_dict[(s.HGNC_ID, s.Assembly)]['Sequence']
#     a, b, _ = find_replacement_substring(ref, s.Sequence)
#     return pd.Series({'mut_start': a, 'mut_end': b})

# clinvar_mut[['mut_start', 'mut_end']] = clinvar_mut.progress_apply(_mut_span, axis=1)


# def overlap_hits(uniprot_id, s, e):
#     hits = []
#     for ipr_id, descr, s0, e0 in ipr_by_uniprot.get(uniprot_id, ()):
#         if e0 >= s and e >= s0:       # interval overlap
#             hits.append(
#                 {'interpro_id': ipr_id,
#                  'description': descr,
#                  'start': int(s0),
#                  'end': int(e0)}
#             )
#     return hits

# iters = zip(
#     clinvar_mut['uniprot_id'],
#     clinvar_mut['mut_start'],
#     clinvar_mut['mut_end'],
# )

# clinvar_mut['interpro_overlap'] = [
#     overlap_hits(uid, s, e)
#     for uid, s, e in tqdm(iters,
#                           total=len(clinvar_mut),   # so the bar knows the length
#                           desc="InterPro overlap")   # label for the bar
# ]

# clinvar_mut.to_csv(clinvar_mut_path, index=False)

In [9]:
protein2ipr_human

,uniprot_id,interpro_id,description,source_id,start,end
0,A0A024R161,IPR001623,DnaJ domain,PF00226,49,115
1,A0A024R161,IPR001623,DnaJ domain,PR00625,51,69
2,A0A024R161,IPR001623,DnaJ domain,PR00625,69,84
3,A0A024R161,IPR001623,DnaJ domain,PR00625,96,116
4,A0A024R161,IPR001623,DnaJ domain,PS50076,49,124
...,...,...,...,...,...,...
241009,X6R8R1,IPR035892,C2 domain superfamily,G3DSA:2.60.40.150,197,322
241010,X6R8R1,IPR035892,C2 domain superfamily,G3DSA:2.60.40.150,323,470
241011,X6R8R1,IPR035892,C2 domain superfamily,SSF49562,200,322
241012,X6R8R1,IPR035892,C2 domain superfamily,SSF49562,322,460


In [10]:
protein2ipr_human[protein2ipr_human['uniprot_id'] == 'P04637']

,uniprot_id,interpro_id,description,source_id,start,end
37969,P04637,IPR002117,p53 tumour suppressor family,PR00386,116,142
37970,P04637,IPR002117,p53 tumour suppressor family,PR00386,158,179
37971,P04637,IPR002117,p53 tumour suppressor family,PR00386,213,234
37972,P04637,IPR002117,p53 tumour suppressor family,PR00386,236,258
37973,P04637,IPR002117,p53 tumour suppressor family,PR00386,264,286
37974,P04637,IPR002117,p53 tumour suppressor family,PR00386,326,350
37975,P04637,IPR002117,p53 tumour suppressor family,PTHR11447,3,369
37976,P04637,IPR008967,"p53-like transcription factor, DNA-binding dom...",SSF49417,97,287
37977,P04637,IPR010991,"p53, tetramerisation domain",PF07710,319,357
37978,P04637,IPR011615,"p53, DNA-binding domain",PF00870,100,288


In [11]:
clinvar_mut

,HGNC_ID,Assembly,Chromosome,Start,Stop,Type,ReferenceAlleleVCF,AlternateAlleleVCF,Sequence,Name
0,HGNC:22197,GRCh37,7,4820844,4820847,Indel,GGAT,TGCTGTAAACTGTAACTGTAAA,MFSAGAESLLHQAREIQDEELKKFCSLL,NM_014855.3(AP5Z1):c.80_83delinsTGCTGTAAACTGTA...
1,HGNC:4263,GRCh37,5,42565977,42565977,single nucleotide variant,A,G,VDLWQLLLTLALAGSSDAFSGSEATAAILSRAPWSLQSVNPGLKTN...,NM_000163.5(GHR):c.1A>G (p.Met1Val)
2,HGNC:6597,GRCh37,5,38506064,38506067,Deletion,GCATT,G,MMDIYVCLKRPSWMVDNKRMRTASNFQWLLSTFILLYLMNQVNSQK...,NM_001127671.2(LIFR):c.1231_1234del (p.Asn411fs)
3,HGNC:6597,GRCh37,5,38499664,38499665,Duplication,G,GT,MMDIYVCLKRPSWMVDNKRMRTASNFQWLLSTFILLYLMNQVNSQK...,NM_001127671.2(LIFR):c.1621dup (p.Thr541fs)
4,HGNC:25801,GRCh37,5,37198926,37198926,single nucleotide variant,G,A,MEIRLEILTSTGIKQKKPWPRVSWLGKEKEAVFLLDDKFINEINLL...,NM_001384732.1(CPLANE1):c.3550C>T (p.Arg1184Cys)
...,...,...,...,...,...,...,...,...,...,...
1996527,HGNC:10518,GRCh38,6,33268516,33268516,single nucleotide variant,C,A,MAAAATMAAAARELVLRAGTSDMEEEEGPLAGGPGLQEPLQLGELD...,NM_022553.6(VPS52):c.682G>T (p.Asp228Tyr)
1996528,HGNC:12857,GRCh38,7,100779559,100779559,single nucleotide variant,C,T,MVPPVWTLLLLVGAALFRKEKPPDQKLVVRSSRDNYVLTQCDFEDD...,NM_003386.3(ZAN):c.6431C>T (p.Ala2144Val)
1996529,HGNC:14124,GRCh38,16,93324,93324,single nucleotide variant,A,G,MRDNTSPISVILVSSGSRGNKLLFRYPFQRSQEHPASQTSKPRSRY...,NM_001077350.3(NPRL3):c.926T>C (p.Val309Ala)
1996530,HGNC:21661,GRCh38,6,30987377,30987377,single nucleotide variant,G,A,MKMQKGNVLLMFGLLLHLEAATNSNETSTSANTGSSVISSGASTAT...,NM_001010909.5(MUC21):c.1202G>A (p.Gly401Glu)


In [12]:
clinvar_ref

,HGNC_ID,Assembly,Sequence
0,HGNC:22197,GRCh37,MFSAGAESLLHQAREIQDEELKKFCSRICKLLQAEDLGPDTLDSLQ...
1,HGNC:30613,GRCh37,MRHNQMCCETPPTVTVYVKSGSNRSHQPKKPITLKRPICKDNWQAF...
2,HGNC:7993,GRCh37,MSEGESQTVLSSGSDPKVESSSSAPGLTSVSPPVTSTTSAASPEEE...
3,HGNC:40022,GRCh37,MREGKRGPPCILSFRGTLERVEAQWELEAQGPGQCPAPLGDPASTT...
4,HGNC:4186,GRCh37,MSAAFPPSLMMMQRPLGSSTAFSIDSLIGSPPQPSPGHFVYTGYPM...
...,...,...,...
18756,HGNC:9246,GRCh38,MMCEVMPTINEDTPMSQRGSQSSGSDSDSHFEQLMVNMLDERDRLL...
18757,HGNC:20157,GRCh38,MADVDPDTLLEWLQMGQGDERDMQLIALEQLCMLLLMSDNVDRCFE...
18758,HGNC:20362,GRCh38,MSATSWFLVSSSGARHRLPRELIFVGREECELMLQSRSVDKQHAVI...
18759,HGNC:4383,GRCh38,MARSLTWRCCPWCLTEDEKAAARVDQEINRILLEQKKQDRGELKLL...


In [5]:
clinvar_ref_dict

{('HGNC:22197',
  'GRCh37'): {'Sequence': 'MFSAGAESLLHQAREIQDEELKKFCSRICKLLQAEDLGPDTLDSLQRLFLIISATKYSRRLEKTCVDLLQATLGLPACPEQLQVLCAAILREMSPSDSLSLAWDHTQNSRQLSLVASVLLAQGDRNEEVRAVGQGVLRALESRQPEGPSLRHLLPVMAKVVVLSPGTLQEDQATLLSKRLVDWLRYASLQQGLPHSGGFFSTPRARQPGPVTEVDGAVATDFFTVLSSGHRFTDDQWLNVQAFSMLRAWLLHSGPEGPGTLDTDDRSEQEGSTLSVISATSSAGRLLPPRERLREVAFEYCQRLIEQSNRRALRKGDSDLQKACLVEAVLVLDVLCRQDPSFLYRSLSCLKALHGRVRGDPASVRVLLPLAHFFLSHGEAAAVDSEAVYQHLFTRIPVEQFHSPMLAFEFIQFCRDNLHLFSGHLSTLRLSFPNLFKFLAWNSPPLTSEFVALLPALVDAGTALEMLHALLDLPCLTAVLDLQLRSAPAASERPLWDTSLRAPSCLEAFRDPQFQGLFQYLLRPKASGATERLAPLHQLLQPMAGCARVAQCAQAVPTLLQAFFSAVTQVADGSLINQLALLLLGRSDSLYPAPGYAAGVHSVLSSQFLALCTLKPSLVVELARDLLEFLGSVNGLCSRASLVTSVVWAIGEYLSVTYDRRCTVEQINKFFEALEALLFEVTQCRPSAALPRCPPQVVTVLMTTLTKLASRSQDLIPRASLLLSKMRTLAHSPATSSTHSEEGAEAIRTRATELLTLLKMPSVAQFVLTPSTEVCSPRYHRDANTALPLALRTVSRLVEREAGLMPG'},
 ('HGNC:30613',
  'GRCh37'): {'Sequence': 'MRHNQMCCETPPTVTVYVKSGSNRSHQPKKPITLKRPICKDNWQAFEKNTHNNNKSKRPKGPCLVIQRQDMTAFFKLFDDDLIQDFLWMDCCCKIADKYLLAM

In [ ]:
# annotate mutation type 
mutations = pd.read_csv('/data/macaulay/oop/mutation_data/name_to_sequence.csv')
print(len(mutations))

clinvar_mut = pd.read_csv('/data/dandreas/clinvar/mut_sequences_clean.csv')
overlapping_columns = set(clinvar_mut.columns) & set(mutations.columns)
print(f'Overlapping columns: {overlapping_columns}')
clinvar_mut_full = pd.merge(
    clinvar_mut,
    mutations,
    on = list(overlapping_columns),
    how = 'left',
)
print(len(clinvar_mut_full))

assert len(clinvar_mut_full) == len(clinvar_mut), "ClinVar mutation rows lost during merge!"

def classify_mutations(df, ref = clinvar_ref_dict):

    def levenshtein(row, ref_dict):
        mut_seq = str(row["Sequence"]).strip()
        ref_seq = str(ref_dict[(row.HGNC_ID, row.Assembly)]["Sequence"]).strip()

        out = find_replacement_substring(mut_seq, ref_seq)
        if out == (None, None, None):
            return 0  # identical
        start, end_mut, end_ref = out

        L_ref = end_ref - start
        L_mut = end_mut - start
        return max(L_ref, L_mut)

    def classify_vcf(ref: str, alt: str) -> str:
        """Return Frame_Shift_* | In_Frame_* | SNP"""
        # Simple single‐base swap
        if len(ref) == 1 and len(alt) == 1:
            return "Substitution"

        # Insertion vs deletion
        if len(ref) < len(alt):
            diff = len(alt) - len(ref)
            return "In_Frame_Insertion" if diff % 3 == 0 else "Frame_Shift_Insertion"

        if len(ref) > len(alt):
            diff = len(ref) - len(alt)
            return "In_Frame_Deletion" if diff % 3 == 0 else "Frame_Shift_Deletion"

        # Same length but not single-bp   → treat as SNP-style substitution
        return "Substitution" # Multiple Nucleotide Polymorphism
    
    def refine_with_nonstop(row):
        cat = row["MutationType"]
        if cat.startswith("Frame_Shift"):
            return cat

        mut_seq = str(row["Sequence"]).strip()
        ref_seq = str(ref[(row.HGNC_ID, row.Assembly)]["Sequence"]).strip()

        start, end_mut, end_ref = find_replacement_substring(mut_seq, ref_seq)

        if (start, end_mut, end_ref) == (None, None, None):
            return cat

        # shorter mutant, no common suffix (i.e., truncation right at the boundary)
        if end_mut < end_ref and end_mut == len(mut_seq):
            return "Nonsense"

        return cat
    
    def simplify_interpro(row):
        ipros = set()
        for d in ast.literal_eval(row['interpro_overlap']):
            ipros.add((d['interpro_id'], d['description']))
        return list(ipros)

    def _norm(s: str) -> str:
        # normalize for grouping
        return " ".join(str(s).strip().lower().split())

    def _rank_short_label(desc: str):
        """Higher tuple wins. Prefer specific domain-like labels over broad ones."""
        d = desc.lower()
        return (
            "superfamily" not in d,     # True > False
            "conserved site" not in d,  # True > False
            "family" not in d,          # True > False
            -len(desc)                  # shorter is better
        )

    def simplify_interpro_row(row, interpro_descriptions):
        # parse overlap (stringified list of dicts) → list[dict]
        try:
            items = ast.literal_eval(row.get("interpro_overlap", "[]"))
            if not isinstance(items, list):
                items = []
        except Exception:
            items = []

        # group by canonical description
        chosen = {}  # canon_desc -> (id, short_label)
        for d in items:
            iid = d.get("interpro_id")
            short = (d.get("description") or "").strip()
            if not iid:
                continue
            canon = _norm(interpro_descriptions.get(iid, short))
            cand = (iid, short)
            prev = chosen.get(canon)
            if prev is None or _rank_short_label(short) > _rank_short_label(prev[1]):
                chosen[canon] = cand

        # stable, readable output
        out = list(chosen.values())
        out.sort(key=lambda t: (t[1].lower(), t[0]))  # sort by short label, then ID
        return out

    df = (
        df.assign(
            MutationType=lambda d: d.progress_apply(
                lambda r: classify_vcf(r["ReferenceAlleleVCF"], r["AlternateAlleleVCF"]),
                axis=1,
            )
        )
        .assign(
            MutationType=lambda d: d.progress_apply(refine_with_nonstop, axis=1)
        )
        .assign(
            Levenshtein=lambda d: d.progress_apply(
                lambda r: levenshtein(r, ref), axis=1
            )
        )
        # remove start/stop from interpro_overlap
        .assign(
            interpro_affected = lambda d: d.progress_apply(
                simplify_interpro, axis=1
            )
        )
        # consolidate interpro domains with identical descriptions
        .assign(
        interpro_affected=lambda d: d.progress_apply(
            lambda r: simplify_interpro_row(r, interpro_descriptions), axis=1
        )
    )
    )
    return df

tqdm.pandas()
clinvar_mut_full = classify_mutations(clinvar_mut_full)
clinvar_mut_full.to_csv('/data/dandreas/clinvar/clinvar_mut_full.csv', index=False)

# filter GENIE genes
somatt_mutations = pd.read_pickle('/home/dandreas/SomaticMutationLLM/data_processing/consolidated_data.pkl')
somatt_mutations = pd.DataFrame(somatt_mutations)
somatt_mutations.columns = ['start','end','chrom','build','variant_class','isoforms','hgncId','gene_start','gene_end','ref_allele','strand','mutation_value','barcode','ref_count','alt_count','assay','name']
clinvar_cancer_mutations = clinvar_mut_full[clinvar_mut_full['HGNC_ID'].isin(somatt_mutations['hgncId'].unique())]

clinvar_cancer_mutations.to_csv('/data/dandreas/clinvar/clinvar_cancer_mutations.csv', index=False)
print(len(clinvar_cancer_mutations))

patho = ['Pathogenic', 'Likely pathogenic', 'Pathogenic/Likely pathogenic', 'Benign', 'Likely benign', 'Benign/Likely benign']
clinvar_cancer_mutations_certain_pathogenicity = clinvar_cancer_mutations[clinvar_cancer_mutations['ClinicalSignificance'].isin(patho)]
clinvar_cancer_mutations_certain_pathogenicity.to_csv('/data/dandreas/clinvar/clinvar_cancer_mutations_certain_pathogenicity.csv', index=False)
print(len(clinvar_cancer_mutations_certain_pathogenicity))

/tmp/ipykernel_3565998/2956733674.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations = pd.read_csv('/data/macaulay/oop/mutation_data/name_to_sequence.csv')


7107850
Overlapping columns: {'HGNC_ID', 'Type', 'Stop', 'AlternateAlleleVCF', 'Assembly', 'Start', 'ReferenceAlleleVCF', 'Chromosome', 'Name'}
1996532


100%|██████████| 1996532/1996532 [05:56<00:00, 5592.99it/s] 


496435


In [33]:
word_count = 0
for i,j in clinvar_mut.interpro_affected.iloc[0]:
    x = f"Region: {j}, Description: {interpro_descriptions[i]}"
    word_count += len(x.split())
    print(x)
print(f'word count: {word_count}')

Region: Fibronectin type III, Description: Fibronectin is a dimeric glycoprotein composed of disulfide-linked subunits with a molecular weight of 220-250kDa each. It is involved in cell adhesion, cell morphology, thrombosis, cell migration, and embryonic differentiation. Fibronectin is a modular protein composed of homologous repeats of three prototypical types of domains known as types I, II, and III. Fibronectin type-III (FN3) repeats are both the largest and the most common of the fibronectin subdomains. Domains homologous to FN3 repeats have been found in various animal protein families including other extracellular-matrix molecules, cell-surface receptors, enzymes, and muscle proteins. Structures of individual FN3 domains have revealed a conserved β-sandwich fold with one β-sheet containing four strands and the other sheet containing three strands (see for example ). This fold is topologically very similar to that of Ig-like domains, with a notable difference being the lack of a c

In [37]:
print(clinvar_mut.PhenotypeList.iloc[0])
print(clinvar_mut.ClinicalSignificance.iloc[0])

not provided|Stüve-Wiedemann syndrome 1
Pathogenic


In [43]:
import ast
import numpy as np
import pandas as pd

# --- helpers ---
def _ensure_list(x):
    """interpro_affected may be a list of (id, short) tuples or a string repr."""
    if isinstance(x, list):
        return x
    if pd.isna(x):
        return []
    try:
        v = ast.literal_eval(str(x))
        return v if isinstance(v, list) else []
    except Exception:
        return []

def _row_counts(pairs, interpro_descriptions):
    """
    pairs: list[(interpro_id, short_desc)]
    returns: (n_ids, total_word_count)
    """
    pairs = _ensure_list(pairs)
    n_ids = len(pairs)
    wc = 0
    for iid, short in pairs:
        long = interpro_descriptions.get(iid, "")
        x = f"Region: {short}, Description: {long}"
        wc += len(x.split())
    return n_ids, wc

# --- compute per-row metrics for the three MutationTypes of interest ---
target_types = {"Substitution", "In_Frame_Insertion", "In_Frame_Deletion"}
df = clinvar_mut[clinvar_mut["MutationType"].isin(target_types)].copy()

df[["n_interpro_ids", "interpro_word_count"]] = df["interpro_affected"].apply(
    lambda v: pd.Series(_row_counts(v, interpro_descriptions))
)

# --- distribution summaries ---
# 1) basic stats by MutationType
summary = (
    df.groupby("MutationType")[["n_interpro_ids", "interpro_word_count"]]
      .agg(["count", "mean", "median", "min", "max", "std"])
      .sort_index()
)

# 2) discrete distribution of "number of affected ids"
n_ids_dist = (
    df.groupby("MutationType")["n_interpro_ids"]
      .value_counts()
      .unstack(fill_value=0)     # columns = 0 ids, 1 id, 2 ids, ...
      .sort_index(axis=1)
      .sort_index()
)

# 3) word-count distribution (quantile bins per type, for comparability)
#    if you prefer fixed-width bins, replace qcut with pd.cut and set bins.
def _wc_hist(g, q=10):
    binned = pd.qcut(g["interpro_word_count"], q=q, duplicates="drop")
    return binned.value_counts().sort_index()

wordcount_dist = (
    df.groupby("MutationType", group_keys=False)
      .apply(_wc_hist, q=25)
      .rename("rows")
      .reset_index()
)

# --- (optional) quick peek ---
print("\n=== Per-type summary stats ===")
print(summary)
print("\n=== Distribution of # InterPro IDs per row ===")
print(n_ids_dist)
print("\n=== Word-count decile distribution (rows per bin) ===")
print(wordcount_dist)



=== Per-type summary stats ===
                   n_interpro_ids                                     \
                            count      mean median min max       std   
MutationType                                                           
In_Frame_Deletion            1569  2.073295    2.0   0   9  1.725146   
In_Frame_Insertion            662  1.604230    1.0   0  10  1.784243   
Substitution                42548  2.021599    2.0   0  16  1.629798   

                   interpro_word_count                               \
                                 count        mean median min   max   
MutationType                                                          
In_Frame_Deletion                 1569  353.188655  250.0   0  2089   
In_Frame_Insertion                 662  267.024169  124.0   0  1715   
Substitution                     42548  347.203817  243.0   0  3323   

                                
                           std  
MutationType                    
In_Frame_

/tmp/ipykernel_3565998/806130374.py:64: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("MutationType", group_keys=False)


In [17]:
import sys
print(sys.path)

['/home/dandreas/.conda/envs/esm3_peft/lib/python310.zip', '/home/dandreas/.conda/envs/esm3_peft/lib/python3.10', '/home/dandreas/.conda/envs/esm3_peft/lib/python3.10/lib-dynload', '', '/home/dandreas/.local/lib/python3.10/site-packages', '/home/dandreas/.conda/envs/esm3_peft/lib/python3.10/site-packages', '/home/dandreas/.conda/envs/esm3_peft/lib/python3.10/site-packages/setuptools/_vendor']


In [ ]:
# # 2 is the length of the empty list
# nonzeros = clinvar_mut[clinvar_mut['interpro_overlap'].str.len() > 2].shape[0]
# zeros = clinvar_mut[clinvar_mut['interpro_overlap'].str.len() == 2].shape[0]
# print('All clinvar')
# print(f'Non-zero overlaps: {nonzeros}, Zero overlaps: {zeros}')

# nonzeros = clinvar_cancer_mutations_full[clinvar_cancer_mutations_full['interpro_overlap'].str.len() > 2].shape[0]
# zeros = clinvar_cancer_mutations_full[clinvar_cancer_mutations_full['interpro_overlap'].str.len() == 2].shape[0]
# print('Clinvar cancer genes')
# print(f'Non-zero overlaps: {nonzeros}, Zero overlaps: {zeros}')

All clinvar
Non-zero overlaps: 1697738, Zero overlaps: 298794
Clinvar cancer genes
Non-zero overlaps: 423576, Zero overlaps: 72859


In [ ]:
# mutations = pd.read_csv('/data/macaulay/oop/mutation_data/name_to_sequence.csv')
# mutations_somatic = mutations[mutations['OriginSimple']=='somatic']
# mutations_somatic_genes = mutations_somatic['HGNC_ID'].unique()
# print(f'Number of somatic genes in ClinVar: {len(mutations_somatic_genes)}')
# somatt_genes = somatt_mutations['hgncId'].unique()
# print(f'Number of SomAtt genes: {len(somatt_genes)}')
# overlap = set(mutations_somatic_genes) & set(somatt_genes)
# print(f'Overlap between somatic mutations and SomAtt genes: {len(overlap)}')

/tmp/ipykernel_866361/242356449.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  mutations = pd.read_csv('/data/macaulay/oop/mutation_data/name_to_sequence.csv')


Number of somatic genes in ClinVar: 1379
Number of SomAtt genes: 1445
Overlap between somatic mutations and SomAtt genes: 452


# Uniprot Text

In [ ]:
from Bio import SwissProt
from io import StringIO
from tqdm import tqdm
from saveAndLoad import pickleSave, pickleLoad
# https://github.com/biopython/biopython/pull/4028/commits/a1ea613845732267175aa084e00af69182e6b2b5

acc2record = {}
path = "/data/macaulay/oop/protein_dataset/uniprot_sprot.dat"

# 1) Raw record blocks
with open(path) as f:
    text = f.read()
blocks = text.split("//\n")

# SKIP = ['FT', 'DR', 'RN', 'RP', 'RC', 'RX', 'RA', 'RT', 'RL', 'RG']
# 2) Parse each block in isolation, skip the ones that crash
not_block = 0
not_read = 0

for block in tqdm(blocks, total = len(blocks)):

    lines = block.splitlines()
    # filtered = [l for l in lines if l[:2] not in SKIP]
    unfiltered = [l for l in lines]
    # if not filtered: 
    #     not_block += 1
    #     continue
    
    block = "\n".join(unfiltered) + "\n//\n"
    # SwissProt.read(StringIO(block))  # just to check if it parses

    # clean_block = "\n".join(filtered) + "\n//\n"
    try:
        # rec = SwissProt.read(StringIO(clean_block))
        rec = SwissProt.read(StringIO(block))
    except: 
        not_read += 1
        continue

    # 3) Map every accession string to that parsed record
    for acc in rec.accessions:
        acc2record[acc] = rec
print(f"Skipped {not_block} empty blocks and {not_read} unreadable records")
pickleSave(acc2record, '/data/dandreas/clinvar/', 'uniprot_sprot_acc2record.pkl')

100%|██████████| 572971/572971 [04:28<00:00, 2131.53it/s]

Skipped 0 empty blocks and 1 unreadable records


In [ ]:
#2m 37s load
acc2record = pickleLoad('/data/dandreas/clinvar/uniprot_sprot_acc2record.pkl')

loading data from /data/dandreas/for_macaulay/uniprot_sprot_acc2record.pkl


In [93]:
# grab any one record
rec = next(iter(acc2record.values()))

# list all available fields and methods
print([i for i in dir(rec) if not i.startswith('__')])

acc2record['P04637'].comments
##comments
##description (contains RecName, AltName)
##gene_name (TP53, synonyms)
##keywords

#created (date)
#annotation_update (date)
#accessions (other accs)
#cross_references []
#data_class (Reviewed)
#entry_name (P53_HUMAN)
#features [REMOVED]
#host_organism
#host_taxonomy_id
#molecule_type (empty)
#organelle ('')
#organism (Homo sapiens)
#organism_classification (Eukaryota; Metazoa; etc.)
#protein_existence (1)
#references [REMOVED]
#seqinfo (393, 43653, AD5C149FD8106131)
#sequence (MEEPQSDPSV...)
#sequence_length (393)
#sequence_update (date)
#taxonomy_id (9606)

['accessions', 'annotation_update', 'comments', 'created', 'cross_references', 'data_class', 'description', 'entry_name', 'features', 'gene_name', 'host_organism', 'host_taxonomy_id', 'keywords', 'molecule_type', 'organelle', 'organism', 'organism_classification', 'protein_existence', 'references', 'seqinfo', 'sequence', 'sequence_length', 'sequence_update', 'taxonomy_id']


['FUNCTION: Multifunctional transcription factor that induces cell cycle arrest, DNA repair or apoptosis upon binding to its target DNA sequence (PubMed:11025664, PubMed:12524540, PubMed:12810724, PubMed:15186775, PubMed:15340061, PubMed:17317671, PubMed:17349958, PubMed:19556538, PubMed:20673990, PubMed:20959462, PubMed:22726440, PubMed:24051492, PubMed:24652652, PubMed:35618207, PubMed:36634798, PubMed:38653238, PubMed:9840937). Acts as a tumor suppressor in many tumor types; induces growth arrest or apoptosis depending on the physiological circumstances and cell type (PubMed:11025664, PubMed:12524540, PubMed:12810724, PubMed:15186775, PubMed:15340061, PubMed:17189187, PubMed:17317671, PubMed:17349958, PubMed:19556538, PubMed:20673990, PubMed:20959462, PubMed:22726440, PubMed:24051492, PubMed:24652652, PubMed:38653238, PubMed:9840937). Negatively regulates cell division by controlling expression of a set of genes required for this process (PubMed:11025664, PubMed:12524540, PubMed:128

In [95]:
from Bio import SwissProt
from io import StringIO
from tqdm import tqdm
path = "/data/macaulay/oop/protein_dataset/uniprot_sprot.dat"

# 1) Raw record blocks
with open(path) as f:
    text = f.read()
blocks = text.split("//\n")

for i in blocks: 
    if 'AC   P04637' in i:
        lines = i.splitlines()
        unfiltered = [l for l in lines]
        block = "\n".join(unfiltered) + "\n//\n"
        out = SwissProt.read(StringIO(block))  # just to check if it parses
        # print(i)
        # #write this to a file
        # with open("output.txt", "w") as f:
        #     f.write(i.strip())
        # break


In [2]:
print(out.features[10])

type: REGION
location: [115:292]
qualifiers:
    Key: evidence, Value: ECO:0000250
    Key: note, Value: Interaction with AXIN1



In [56]:
record = acc2record.get('P04637')

def get_diseases(record):
    """
    For each DISEASE comment in acc2record[acc], returns
    "<description> - <note>"
    If Note only, returns "Unnamed disease - <note>".
    """

    out = []
    count = 0
    for block in get_comments(record, 'DISEASE'):
        if block.upper().startswith("NOTE="):
            note = block[len("Note="):].strip()
            out.append(f"Note - {note}")
            count += 1
            continue

        if " Note=" in block:
            main_part, note_part = block.split(" Note=", 1)
            note = note_part.strip()
            print(note)
        else:
            main_part, note = block, ""

        combined = f"{main_part} - {note}"
        out.append(combined)
        count+=1

    return out


get_diseases(record)

The disease is caused by variants affecting the gene represented in this entry
The disease is caused by variants affecting the gene represented in this entry
The gene represented in this entry is involved in disease pathogenesis
The disease is caused by variants affecting the gene represented in this entry
The disease is caused by variants affecting the gene represented in this entry
The disease is caused by variants affecting the gene represented in this entry
Disease susceptibility is associated with variants affecting the gene represented in this entry
The disease is caused by variants affecting the gene represented in this entry


['Note - TP53 is found in increased amounts in a wide variety of transformed cells. TP53 is frequently mutated or inactivated in about 60% of cancers. TP53 defects are found in Barrett metaplasia a condition in which the normally stratified squamous epithelium of the lower esophagus is replaced by a metaplastic columnar epithelium. The condition develops as a complication in approximately 10% of patients with chronic gastroesophageal reflux disease and predisposes to the development of esophageal adenocarcinoma',
 'Esophageal cancer (ESCR): A malignancy of the esophagus. The most common types are esophageal squamous cell carcinoma and adenocarcinoma. Cancer of the esophagus remains a devastating disease because it is usually not detected until it has progressed to an advanced incurable stage. - The disease is caused by variants affecting the gene represented in this entry',
 'Li-Fraumeni syndrome (LFS): An autosomal dominant familial cancer syndrome that in its classic form is defined 

In [ ]:
import re

def clean_identifiers(text: str) -> str:
    """
    1) Remove TAG:VALUE sequences and {...}/[...] blocks
    2) Remove empty parentheses like '(, , )'
    3) Remove any {...} or [...] entirely
    4) Remove spaces before punctuation, fix spacing around punctuation
    5) Collapse repeated punctuation
    6) Trim leading/trailing punctuation or whitespace
    """
    # ----- 1) same as before -----
    text = re.sub(r'\b[A-Z][A-Za-z0-9\-]+:[A-Za-z0-9\-\|,]+\b', '', text)
    text = re.sub(r'\{[^}]*\}', '', text)
    text = re.sub(r'\[[^\]]*\]', '', text)
    text = re.sub(r'\([^A-Za-z0-9]*\)', '', text)
    text = re.sub(r'\s{2,}', ' ', text)
    text = re.sub(r',{2,}', ',', text)
    text = re.sub(r';{2,}', ';', text)
    text = re.sub(r'\.{2,}', '.', text)

    # ----- 2) strip spaces before punctuation -----
    #   " something ." -> " something."
    text = re.sub(r'\s+([.,;:])', r'\1', text)

    # ----- 3) ensure single space after punctuation when needed -----
    #   "word.And" -> "word. And"
    text = re.sub(r'([.,;:])([^\s])', r'\1 \2', text)

    # ----- 4) collapse any ".;" or ":;" or ";," into a single char -----
    text = re.sub(r'\.\s*[;,:]', '.', text)
    text = re.sub(r';\s*[,\.]', ';', text)

    # ----- 5) final trim of leftover edges -----
    text = text.strip(" \t\n\r\f\v.,;:-")

    return text

def get_comments(record, tag):
    """Return cleaned lines for any CC -!- TAG: ... block."""
    out = []
    prefix = f"{tag.upper()}:"
    for block in record.comments:
        for line in block.splitlines():
            if line.upper().startswith(prefix):
                txt = line.split(":", 1)[1].strip()
                out.append(clean_identifiers(txt))
    return out

def parse_gene_name(raw: str):
    """
    Given a string of the form:
      "Name=GENE; Synonyms=SYN1, SYN2, ...;"
    return [GENE, SYN1, SYN2, ...].
    """
    # strip trailing semicolons/spaces
    raw = raw.strip().rstrip(";")
    parts = [p.strip() for p in raw.split(";") if p.strip()]
    name = None
    synonyms = []
    for part in parts:
        if part.startswith("Name="):
            name = part.split("=", 1)[1].strip()
        elif part.startswith("Synonyms="):
            syn_str = part.split("=", 1)[1].strip()
            # split on commas, strip whitespace
            synonyms = [s.strip() for s in syn_str.split(",") if s.strip()]
    result = []
    if name:
        result.append(name)
    result.extend(synonyms)
    return result

def format_gene_name(raw: str) -> str:
    """
    Return a comma‑separated string of the parsed gene name + synonyms.
    """
    return ", ".join(parse_gene_name(raw))


def get_name(record):
    """Parse RecName: Full=...; or fallback to entry_name."""
    desc = record.description or ""
    m = re.search(r"RecName:\s*Full=(.+?);", desc)
    if m: return m.group(1).strip()
    m2 = re.search(r"Full=(.+?)(?:;|$)", desc)
    if m2: return m2.group(1).strip()
    return (desc.strip() or record.entry_name)

def get_interactions(record) -> list[str]:
    pairs = []
    for block in record.comments:
        if block.upper().startswith("INTERACTION:"):
            # find all UniProt‐style accessions in that block
            found = re.findall(r"\b[OPQ][0-9A-Z]{5}\b", block)
            pairs.extend(found)
    pairs = set(list(pairs))
    names = []  
    for acc in pairs:
        rec2 = acc2record.get(acc)
        nm = get_name(rec2) if rec2 else ''
        gene_name = rec2.gene_name if rec2 else ''
        if (nm != '') or (gene_name != ''):
            names.append(f'{clean_identifiers(nm)} ({format_gene_name(gene_name)})')
    return names

def get_diseases(record):
    """
    For each DISEASE comment in acc2record[acc], returns
    "<description> - <note>"
    If Note only, returns "Unnamed disease - <note>".
    """

    out = []
    for block in get_comments(record, 'DISEASE'):
        if block.upper().startswith("NOTE="):
            note = block[len("Note="):].strip()
            out.append(f"Note - {note}")
            continue

        if " Note=" in block:
            main_part, note_part = block.split(" Note=", 1)
            note = note_part.strip()
        else:
            main_part, note = block, ""

        combined = f"{main_part} - {note}"
        out.append(combined)

    return out

# def get_features(record):
#     # print(dir(acc2record['O43299'].features[0]))
#     # print(acc2record['O43299'].features[0].qualifiers['note'])
#     out = []
#     for raw in record.features:


def parse_cofactors(record):
    """Extract only Name and Note from COFACTOR blocks."""
    out = []
    for raw in get_comments(record, "COFACTOR"):
        # split on ';', capture fields
        parts = [p.strip() for p in raw.split(";") if p.strip()]
        d = {}
        for p in parts:
            if p.startswith("Name="):
                d["Name"] = p.split("=",1)[1]
            elif p.startswith("Note="):
                d["Note"] = p.split("=",1)[1]
        if d:
            if "Note" in d:
                out.append(f"{d['Name']}: {clean_identifiers(d['Note'])}")
            else:
                out.append(d["Name"])
    return out

def parse_locations(record):
    """
    Split SUBCELLULAR LOCATION into:
      - general compartments (first CC block)
      - isoform-specific lines
    """
    all_locs = get_comments(record, "SUBCELLULAR LOCATION")
    general = []
    isoforms = []
    for entry in all_locs:
        if entry.startswith("[Isoform"):
            isoforms.append(entry)
        else:
            general.append(entry)
    # clean and join general entries
    gen_clean = []
    for g in general:
        # split on '.' to separate each compartment/note
        parts = [p.strip() for p in re.split(r"\.\s*", g) if p.strip()]
        gen_clean.extend(parts)
    # clean isoform lines
    iso_clean = []
    for iso in isoforms:
        # "[Isoform 1]: Nucleus; Cytoplasm. Note=..."
        m = re.match(r"\[(Isoform [^\]]+)\]:\s*(.*)", iso)
        if not m:
            continue
        iso_id, rest = m.groups()
        # split rest into compartment vs note
        if "Note=" in rest:
            loc_part, note_part = rest.split("Note=",1)
            locs = [l.strip() for l in re.split(r"[.;]\s*", loc_part) if l.strip()]
            note = note_part.strip().rstrip(".")
        else:
            locs = [l.strip() for l in re.split(r"[.;]\s*", rest) if l.strip()]
            note = None

        locs_str = ", ".join(clean_identifiers(l) for l in locs)
        if note:
            iso_clean.append(f"{iso_id}: {locs_str} (note: {clean_identifiers(note)})")
        else:
            iso_clean.append(f"{iso_id}: {locs_str}")
    return gen_clean, iso_clean

def fetch_uniprot_context(accession):
    rec = acc2record.get(accession)
    if rec is None:
        raise KeyError(f"No local record for {accession}")

    sections = {}
    # 1) Name + description
    name = f'{get_name(rec)} ({format_gene_name(rec.gene_name)})'
    sections["NAME"] = clean_identifiers(name)

    # 2) Function
    fn = get_comments(rec, "FUNCTION")
    if fn:
        sections["FUNCTION"] = clean_identifiers(" ".join(fn))

    # 3) Cofactor
    cof = parse_cofactors(rec)
    if cof:
        sections["COFACTOR"] = clean_identifiers("; ".join(cof))

    # 4) Subunit
    sub = get_comments(rec, "SUBUNIT")
    if sub:
        sections["SUBUNIT"] = clean_identifiers("; ".join(sub))

    # 5) Interaction partners
    ix = get_interactions(rec)
    if ix:
        sections["INTERACTIONS"] = clean_identifiers(", ".join(ix))

    # 6) Disease & variants
    dis = get_diseases(rec)
    if dis:
        sections["DISEASES"] = clean_identifiers("; ".join(dis))

    # 7) Tissue specificity / expression
    expr = get_comments(rec, "TISSUE SPECIFICITY")
    if expr:
        sections["EXPRESSION"] = clean_identifiers("; ".join(expr))
        
    # LOCATIONS
    general, isoforms = parse_locations(rec)
    if general:
        sections["LOCATIONS"] = clean_identifiers("; ".join(g for g in general))
    if isoforms:
        sections["ISOFORM LOCATIONS"] = clean_identifiers("\n  ".join(isoforms))

    # 9) Domains
    dom = get_comments(rec, "DOMAIN")
    if dom:
        sections["DOMAINS"] = clean_identifiers("; ".join(dom))

    # 10) PTMs
    ptms = get_comments(rec, "PTM")
    if ptms:
        sections["POST-TRANSLATIONAL MODIFICATIONS"] = clean_identifiers("; ".join(ptms))

    return sections

# Example usage:
context_block = fetch_uniprot_context("P04637")
print(context_block)


{'NAME': 'Cellular tumor antigen p53 (TP53, P53)', 'FUNCTION': 'Multifunctional transcription factor that induces cell cycle arrest, DNA repair or apoptosis upon binding to its target DNA sequence. Acts as a tumor suppressor in many tumor types; induces growth arrest or apoptosis depending on the physiological circumstances and cell type. Negatively regulates cell division by controlling expression of a set of genes required for this process. One of the activated genes is an inhibitor of cyclin-dependent kinases. Apoptosis induction seems to be mediated either by stimulation of BAX and FAS antigen expression, or by repression of Bcl-2 expression. Its pro-apoptotic activity is activated via its interaction with PPP1R13B/ASPP1 or TP53BP2/ASPP2. However, this activity is inhibited when the interaction with PPP1R13B/ASPP1 or TP53BP2/ASPP2 is displaced by PPP1R13L/iASPP. In cooperation with mitochondrial PPIF is involved in activating oxidative stress-induced necrosis; the function is large

In [57]:
context = []
not_found = set()
for i in tqdm(clinvar_ref.itertuples(index=False), total=len(clinvar_ref)):
    uniprot_id = hgnc_uniprot_name_map.get(i.HGNC_ID, None)
    if uniprot_id is None:
        assert False, f"HGNC_ID {i.HGNC_ID} not found in uniprot map"
    try:
        context_dict = fetch_uniprot_context(uniprot_id)
    except KeyError:
        not_found.add((uniprot_id, i.HGNC_ID))
        continue
    context.append({
        'uniprot_id': uniprot_id,
        'assembly': i.Assembly,
        'sequence': i.Sequence,
        'context': context_dict
    })
context_df = pd.DataFrame(context)
print(len(not_found), "uniprot IDs not found in local records")
    

100%|██████████| 18761/18761 [00:10<00:00, 1848.64it/s]


38 uniprot IDs not found in local records


In [ ]:
context_df.to_pickle('/data/dandreas/clinvar/uniprot_context.pkl')
context_df.to_csv('/data/dandreas/clinvar/uniprot_context.csv', index=False)

In [59]:
context_df

,uniprot_id,assembly,sequence,context
0,O43299,GRCh37,MFSAGAESLLHQAREIQDEELKKFCSRICKLLQAEDLGPDTLDSLQ...,"{'NAME': 'AP-5 complex subunit zeta-1 (AP5Z1, ..."
1,Q5MJ70,GRCh37,MRHNQMCCETPPTVTVYVKSGSNRSHQPKKPITLKRPICKDNWQAF...,"{'NAME': 'Speedy protein A (SPDYA, SPDY1, SPY1..."
2,Q9UHY1,GRCh37,MSEGESQTVLSSGSDPKVESSSSAPGLTSVSPPVTSTTSAASPEEE...,{'NAME': 'Nuclear receptor-binding protein (NR...
3,A6NMD0,GRCh37,MREGKRGPPCILSFRGTLERVEAQWELEAQGPGQCPAPLGDPASTT...,{'NAME': 'Interferon-induced transmembrane pro...
4,P52951,GRCh37,MSAAFPPSLMMMQRPLGSSTAFSIDSLIGSPPQPSPGHFVYTGYPM...,"{'NAME': 'Homeobox protein GBX-2 (GBX2)', 'FUN..."
...,...,...,...,...
18718,O75334,GRCh38,MMCEVMPTINEDTPMSQRGSQSSGSDSDSHFEQLMVNMLDERDRLL...,"{'NAME': 'Liprin-alpha-2 (PPFIA2)', 'FUNCTION'..."
18719,Q9ULT8,GRCh38,MADVDPDTLLEWLQMGQGDERDMQLIALEQLCMLLLMSDNVDRCFE...,{'NAME': 'E3 ubiquitin-protein ligase HECTD1 (...
18720,Q9Y4F5,GRCh38,MSATSWFLVSSSGARHRLPRELIFVGREECELMLQSRSVDKQHAVI...,{'NAME': 'Centrosomal protein of 170 kDa prote...
18721,P30679,GRCh38,MARSLTWRCCPWCLTEDEKAAARVDQEINRILLEQKKQDRGELKLL...,{'NAME': 'Guanine nucleotide-binding protein s...


In [ ]:
context_df = pd.read_pickle('/data/dandreas/clinvar/uniprot_context.pkl')
context_df

,uniprot_id,assembly,sequence,context
0,O43299,GRCh37,MFSAGAESLLHQAREIQDEELKKFCSRICKLLQAEDLGPDTLDSLQ...,"{'NAME': 'AP-5 complex subunit zeta-1 (AP5Z1, ..."
1,Q5MJ70,GRCh37,MRHNQMCCETPPTVTVYVKSGSNRSHQPKKPITLKRPICKDNWQAF...,"{'NAME': 'Speedy protein A (SPDYA, SPDY1, SPY1..."
2,Q9UHY1,GRCh37,MSEGESQTVLSSGSDPKVESSSSAPGLTSVSPPVTSTTSAASPEEE...,{'NAME': 'Nuclear receptor-binding protein (NR...
3,A6NMD0,GRCh37,MREGKRGPPCILSFRGTLERVEAQWELEAQGPGQCPAPLGDPASTT...,{'NAME': 'Interferon-induced transmembrane pro...
4,P52951,GRCh37,MSAAFPPSLMMMQRPLGSSTAFSIDSLIGSPPQPSPGHFVYTGYPM...,"{'NAME': 'Homeobox protein GBX-2 (GBX2)', 'FUN..."
...,...,...,...,...
18718,O75334,GRCh38,MMCEVMPTINEDTPMSQRGSQSSGSDSDSHFEQLMVNMLDERDRLL...,"{'NAME': 'Liprin-alpha-2 (PPFIA2)', 'FUNCTION'..."
18719,Q9ULT8,GRCh38,MADVDPDTLLEWLQMGQGDERDMQLIALEQLCMLLLMSDNVDRCFE...,{'NAME': 'E3 ubiquitin-protein ligase HECTD1 (...
18720,Q9Y4F5,GRCh38,MSATSWFLVSSSGARHRLPRELIFVGREECELMLQSRSVDKQHAVI...,{'NAME': 'Centrosomal protein of 170 kDa prote...
18721,P30679,GRCh38,MARSLTWRCCPWCLTEDEKAAARVDQEINRILLEQKKQDRGELKLL...,{'NAME': 'Guanine nucleotide-binding protein s...


In [15]:
context_df[context_df['uniprot_id'] == 'P04637'].iloc[0].context

{'NAME': 'Cellular tumor antigen p53 (TP53, P53)',
 'FUNCTION': 'Multifunctional transcription factor that induces cell cycle arrest, DNA repair or apoptosis upon binding to its target DNA sequence. Acts as a tumor suppressor in many tumor types; induces growth arrest or apoptosis depending on the physiological circumstances and cell type. Negatively regulates cell division by controlling expression of a set of genes required for this process. One of the activated genes is an inhibitor of cyclin-dependent kinases. Apoptosis induction seems to be mediated either by stimulation of BAX and FAS antigen expression, or by repression of Bcl-2 expression. Its pro-apoptotic activity is activated via its interaction with PPP1R13B/ASPP1 or TP53BP2/ASPP2. However, this activity is inhibited when the interaction with PPP1R13B/ASPP1 or TP53BP2/ASPP2 is displaced by PPP1R13L/iASPP. In cooperation with mitochondrial PPIF is involved in activating oxidative stress-induced necrosis; the function is larg

In [ ]:
get_note = lambda x: x.qualifiers['note'] if 'note' in x.qualifiers else None
for i in acc2record['P04637'].features:
    print(i.type)
    print((int(i.location.start), int(i.location.end)))
    print(get_note(i))

CHAIN
(0, 393)
Cellular tumor antigen p53
DNA_BIND
(101, 292)
None
REGION
(0, 320)
Interaction with CCAR2
REGION
(0, 83)
Interaction with HRMT1L2
REGION
(0, 44)
Transcription activation (acidic)
REGION
(49, 96)
Disordered
REGION
(65, 110)
Interaction with WWOX
REGION
(99, 370)
Interaction with HIPK1
REGION
(99, 300)
Required for interaction with ZNF385A
REGION
(112, 236)
Required for interaction with FBXO42
REGION
(115, 292)
Interaction with AXIN1
REGION
(240, 248)
Interaction with the 53BP2 SH3 domain
REGION
(255, 294)
Interaction with E4F1
REGION
(272, 280)
Interaction with DNA
REGION
(281, 325)
Disordered
REGION
(299, 393)
Interaction with CARM1
REGION
(318, 360)
Interaction with HIPK2
REGION
(324, 356)
Oligomerization
REGION
(350, 393)
Disordered
REGION
(358, 363)
Interaction with USP7
REGION
(367, 387)
Basic (repression of DNA-binding)
REGION
(373, 393)
Interaction with MORN3
MOTIF
(16, 25)
TADI
MOTIF
(47, 56)
TADII
MOTIF
(304, 321)
Bipartite nuclear localization signal
MOTIF
(338